In [ ]:
#old one
# !/usr/bin/python3

# Simple ReuVision Viewer - fixed
import struct
import sys
import threading
import logging
import time
import cv2
import numpy as np
from scipy.signal import find_peaks
import configparser
import os
import json
import math
import time
import signal
import atexit
import fcntl

cv2.ocl.setUseOpenCL(False)
cv2.setNumThreads(1)

signal.signal(signal.SIGPIPE, signal.SIG_DFL)    # Exit immediately when stdout is closed (SIGPIPE)

sys.path.append("/var/fgvideo/src")
sys.path.append("/var/fgvideo/src/dist-packages")

from ReuCameraPy.ReuCameraPy import *
from ReuOnvifPy import config

logging.basicConfig(level=logging.INFO, format="[%(levelname).1s] %(message)s")

FRAME_W, FRAME_H = config.get_snapshot_resolution()


# Load JSON config
config_path = os.path.join(os.path.dirname(__file__), 'camera_alignment_config.json')
try:
    with open(config_path, 'r') as f:
        cfg = json.load(f)
except FileNotFoundError:
    logging.error(f"Configuration file not found at {config_path}")
    sys.exit(1)
except json.JSONDecodeError:
    logging.error(f"Error decoding JSON from {config_path}")
    sys.exit(1)

REUCAM_NOTIFY_FIREBORDER = 0x8010
sensor_width = cfg["sensor_width"]
sensor_height = cfg["sensor_height"]
pixel_size = cfg["pixel_size"]
focal_length = cfg["focal_length"]
#FPS = cfg["FPS"]
BOX_SIZE = cfg["BOX_SIZE"]
LED_FREQ = cfg["LED_FREQ"]
FREQ_TOL = cfg["FREQ_TOL"]
CALC_LENGTH = cfg["CALC_LENGTH"]
MARGIN = cfg["MARGIN"]
fourier_peak_threshold = cfg["fourier_peak_threshold"]


frame_width = FRAME_W
frame_height = FRAME_H

pixel_size_eff_x = pixel_size * sensor_width / frame_width
pixel_size_eff_y = pixel_size * sensor_height / frame_height

IFOV_x = 2 * np.degrees(np.arctan(pixel_size_eff_x / (2 * focal_length)))
IFOV_y = 2 * np.degrees(np.arctan(pixel_size_eff_y / (2 * focal_length)))

# ===== Global buffers =====
WI, HE = frame_width // BOX_SIZE+1, frame_height // BOX_SIZE+1
fourier_buff = np.zeros((CALC_LENGTH, HE, WI))
freq_counter = np.zeros((HE, WI))
pixel_count_buff = np.zeros(CALC_LENGTH)


# ===== Global variables =====
mode = 1    # Initial mode - general scan
roi_mode1 = None  # Will be defined in mode 1
MARGIN3_factor= 1
LED_CIRCLE_SIZE=0.05
bbox_mode2 = None # Search area for mode 2
flicker_buff = None
scounter = tcounter = m2counter = 0
top_point, bottom_point = None, None
pixel_buffer = None
frame_idx = 0
# Variables for transition to mode 3
flicker_stable_count = 0
STABLE_FRAMES_THRESHOLD = 5  # Number of frames the contour must remain to transition to mode 3
bbox_mode3 = None  # The rectangle that is transitioned to mode 3
ROI_STABLE_FRAMES = 1
ROI_HISTORY = 10    # How many frames to keep (e.g. 10 = about a third of a second at 30fps)
roi_history = []    # List of detected block masks
failure_count_m3=0
MAX_FAILURES_M3=40
roi_stable_count = 0
frame_idx = 0
flicker_buff = None


# ===== Read FPS from /etc/reucamd.conf =====
fps_conf = configparser.ConfigParser()
fps_conf.read("/etc/reucamd.conf")

FPS = fps_conf.getint("camera", "framerate", fallback=cfg["FPS"])

# ===== Read cross position from dev/fgvideo/etc/reucamd.conf =====
cross_conf_path = os.path.join(os.path.dirname(__file__), '..', 'etc', 'reucamd.conf')




PIXEL_HISTORY = FPS * 2


# -----------------------------
# Reucam Service
# -----------------------------
class ReucamClient(ReuCameraClient):
    def __init__(self, svc):
        super().__init__()
        self.svc = svc

    def cmd_onFrame(self, cmd):
        frame = cmd.frame
        enc = frame.enc
        self.svc.onFrame(frame)

class ReucamService(threading.Thread):
    def __init__(self):
        super().__init__()
        self.daemon = True
        self.client = None
        self.lock = threading.Lock()
        self.I = None
        self.ts = None
        self.frame_event = threading.Event()
        self.bActive = False

    def run(self):
        self.bActive = True
        logging.info("ReucamService: started")
        backoff = 1.0
        while self.bActive:
            client = None
            try:
                client = ReucamClient(self)
                self.client = client
                client.connect(ReuCameraService.SERVER_ADDR)
                logging.info("ReucamService: connected")
                client.cmd_StartStream(3)  # RAW

                # recv_loop may raise when peer closes; catch inside loop via exception below
                client.recv_loop()

            except Exception:
                # log full traceback (important for debugging 'Peer closed' etc.)
                logging.exception("ReucamService error - reconnecting after failure")
                # ensure we don't keep a broken client reference
                try:
                    if client:
                        client.close()
                except Exception:
                    logging.exception("Error while closing client after failure")
                self.client = None

                # exponential backoff with cap
                time.sleep(backoff)
                backoff = min(backoff * 2, 30.0)
                continue
            else:
                # clean shutdown of recv_loop (no exception) -> reset backoff
                backoff = 1.0
                try:
                    if client:
                        client.close()
                except Exception:
                    logging.exception("Error while closing client in normal shutdown")
                self.client = None
            finally:
                # small delay to avoid tight loop if bActive still True
                time.sleep(0.2)

    def onFrame(self, frame):
        # frame may be None or malformed; guard it
        try:
            if not frame:
                return
            if getattr(frame, 'is_raw', False) and len(frame.data) == FRAME_W * FRAME_H * 3 // 2:
                with self.lock:
                    self.I = frame.data
                    self.ts = frame.timestamp
                    self.frame_event.set()
            else:
                # log unexpected frames at debug level
                logging.debug("Received non-raw or unexpected-size frame")
        except Exception:
            logging.exception("Error in onFrame handler")

    def onFireBorder(self, param1, param2):
        # protect against missing/closed client
        try:
            if self.client is None:
                logging.debug("onFireBorder called but client is None (not connected)")
                return
            # do not call into a client that might be closed; wrap in try
            try:
                self.client.cmd_Notify(REUCAM_NOTIFY_FIREBORDER, param1, param2)
            except Exception:
                logging.exception("Failed to send Notify to reucam client")
        except Exception:
            logging.exception("Unexpected error in onFireBorder")


# -----------------------------
# Auxiliary functions
# -----------------------------
#   ======================================================================================================================================================
def convert_yuv_to_bgr(yuv_data, width, height):
    y_size = width * height
    uv_size = y_size // 4
    y = np.frombuffer(yuv_data[0:y_size], dtype=np.uint8).reshape((height, width))
    u = np.frombuffer(yuv_data[y_size:y_size+uv_size], dtype=np.uint8).reshape((height//2, width//2))
    v = np.frombuffer(yuv_data[y_size+uv_size:y_size+2*uv_size], dtype=np.uint8).reshape((height//2, width//2))
    u_up = cv2.resize(u, (width, height), interpolation=cv2.INTER_LINEAR)
    v_up = cv2.resize(v, (width, height), interpolation=cv2.INTER_LINEAR)
    yuv = cv2.merge((y, u_up, v_up))
    bgr = cv2.cvtColor(yuv, cv2.COLOR_YUV2BGR)
    return bgr
#def convert_yuv_to_bgr(yuv_data, width, height):
    #"""
    #Converts a YUV420 frame (byte array) to a BGR image (OpenCV format).
    #"""
    #if yuv_data is None:
    #    return None

    #frame_gray = np.frombuffer(yuv_data[0:width * height], dtype=np.uint8).reshape((height, width))
    #frame = cv2.cvtColor(frame_gray, cv2.COLOR_GRAY2BGR)

    #return frame
#   ======================================================================================================================================================
def get_fft_frequency(signal, fps):
    signal = signal - np.mean(signal)
    fft_vals = np.abs(np.fft.rfft(signal))[1:]
    f_axis = np.linspace(0, fps / 2, len(fft_vals) + 1)[1:]
    if len(fft_vals) == 0:
        return 0
    peaks, _ = find_peaks(fft_vals, height=np.max(fft_vals))
    #if len(peaks):
    #    print(f_axis[peaks[0]])
    if len(peaks) > 0 and fft_vals[peaks[0]] > fourier_peak_threshold * np.median(fft_vals):
        return f_axis[peaks[0]]
    return 0

def process_box(frame, gray, B2, h, w):
    b2 = B2[h*BOX_SIZE:(h+1)*BOX_SIZE, w*BOX_SIZE:(w+1)*BOX_SIZE]

    # ????? ????????? FFT ?????
    z = np.roll(fourier_buff[:, h, w], 1)      # ???? np.roll ?? ndimage.shift
    z[0] = np.sum(b2)
    fourier_buff[:, h, w] = z

    # FFT
    freq = get_fft_frequency(fourier_buff[:, h, w], FPS)
    #print(freq)
    # ????? ?????? (?-ROI ?? ??????)
    if abs(freq - LED_FREQ) < FREQ_TOL:
        # ?????????: ????? ???? ????? ??? freq_counter ??????? ???????
        freq_counter[h, w] += 1
        x1 = w * BOX_SIZE
        y1 = h * BOX_SIZE
        x2 = min((w + 1) * BOX_SIZE, frame_width)
        y2 = min((h + 1) * BOX_SIZE, frame_height)
        return (x1, y1, x2, y2), freq

    # ?? ??? ????? � ??? ?????? ????? ??
    freq_counter[h, w] = 0
    return None, None
    
    


#! =============== DIFFERENCE: CELL 2 HAS thresh=100 (vs 140 in Cell 1) ===============
def process_flicker_pixels(frame, roi, flicker_buff, frame_count, buffer_len=10, thresh=100):
    global flicker_stable_count, mode, bbox_mode3
    if roi is None:
        return None, flicker_buff, frame_count, bbox_mode3
#! =============== DIFFERENCE: CELL 2 HAS thresh=100 (vs 140 in Cell 1) ===============

    x1, y1, x2, y2 = roi
    x1 = max(0, x1)
    y1 = max(0, y1)
    x2 = min(frame.shape[1]-1, x2)
    y2 = min(frame.shape[0]-1, y2)

    if x2 <= x1 or y2 <= y1:
        return None, flicker_buff, frame_count, bbox_mode3
    roi_gray = cv2.cvtColor(frame[y1:y2, x1:x2], cv2.COLOR_BGR2GRAY)
    h, w = roi_gray.shape

    if flicker_buff is None or flicker_buff.shape[1:] != (h, w):
        flicker_buff = np.zeros((buffer_len, h, w), dtype=np.uint8)
        frame_count = 0
        flicker_stable_count = 0

    flicker_buff = np.roll(flicker_buff, -1, axis=0)
    flicker_buff[-1] = roi_gray
    frame_count += 1

    mask = np.zeros((h, w), dtype=np.uint8)
    if frame_count > buffer_len:
        diff = flicker_buff.max(axis=0) - flicker_buff.min(axis=0)
        mask[diff > thresh] = 200
        #! =============== DIFFERENCE: CELL 2 USES DYNAMIC MARGIN3 CALCULATION ===============
        #MARGIN3=25
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            all_points = np.vstack(contours)
            x, y, w_box, h_box = cv2.boundingRect(all_points)

            MARGIN3 = int(MARGIN3_factor * max(w_box, h_box))  # Dynamic calculation
            x_margin = max(0, x1 + x - MARGIN3)
            y_margin = max(0, y1 + y - MARGIN3)
            x2_margin = min(frame.shape[1], x1 + x + w_box + MARGIN3)
            y2_margin = min(frame.shape[0], y1 + y + h_box + MARGIN3)
            cv2.rectangle(frame, (x1+x, y1+y), (x1+x+w_box, y1+y+h_box), (0, 0, 255), 2)
        #! =============== DIFFERENCE: CELL 2 USES DYNAMIC MARGIN3 CALCULATION ===============
            flicker_stable_count += 1
            
                       
            if flicker_stable_count >= STABLE_FRAMES_THRESHOLD:
               
               
                #bbox_mode3 = (x1+x, y1+y, w_box, h_box)
                bbox_mode3 = (x_margin, y_margin,x2_margin - x_margin, y2_margin - y_margin)
                tracker.init(frame, bbox_mode3)
                logging.info("tracker.init(frame, bbox_mode3)")
                logging.info(f"bbox_mode3={bbox_mode3}")
                mode = 3
        else:
            flicker_stable_count = 0
            bbox_mode3 = None
            mode = 1

    return mask, flicker_buff, frame_count, bbox_mode3

def convert_bboxes_to_reucam_params(bbox):
    """
    Converts a bounding box to ReuCam parameters safely.
    bbox = [x0, y0, x1, y1]
    Returns param1, param2 (packed)
    """

    # Scale coordinates to FRAME_W / FRAME_H
    x0 = math.ceil(bbox[0] * FRAME_W / frame_width)
    x1 = math.ceil(bbox[2] * FRAME_W / frame_width)
    y0 = math.ceil(bbox[1] * FRAME_H / frame_height)
    y1 = math.ceil(bbox[3] * FRAME_H / frame_height)

    w = x1 - x0
    h = y1 - y0

    # If frame is large, reduce w/h
    if frame_width > 640:
        w = math.ceil(w / 2)
        h = math.ceil(h / 2)

    # Clamp values to valid ranges
    x0 = max(0, min(65535, x0))
    y0 = max(0, min(65535, y0))  # ushort range
    w = max(0, min(255, w))      # B range
    h = max(0, min(255, h))      # B range

    param1 = x0
    try:
        param2 = struct.pack('HBB', y0, w, h)
    except struct.error as e:
        logging.error(f"struct.pack failed: y0={y0}, w={w}, h={h}")
        param2 = struct.pack('HBB', 0, 0, 0)

    param2 = struct.unpack('i', param2)[0]

    return param1, param2

# Set stdin to non-blocking once at startup
fd = sys.stdin.fileno()
fl = fcntl.fcntl(fd, fcntl.F_GETFL)
fcntl.fcntl(fd, fcntl.F_SETFL, fl | os.O_NONBLOCK)

def check_stdin():
    try:
        line = sys.stdin.readline()
        if line:
            return line.strip()
    except Exception:
        return None
    return None

last_fireborder = None
last_fire_time = 0

# -----------------------------
# Main
# -----------------------------
if __name__ == "__main__":
    reucam = ReucamService()
    reucam.start()
    def cleanup():
        try:
            reucam.onFireBorder(0, 0)
        except Exception:
            pass  # ignore errors if reucam is not available yet
   
    # Register cleanup for normal exit
    atexit.register(cleanup)
   
    # Handle termination signals
    def handle_signal(signum, frame):
        cleanup()
        sys.exit(0)
   
    signal.signal(signal.SIGINT, handle_signal)   # CTRL+C
    signal.signal(signal.SIGTERM, handle_signal)  # kill command
    signal.signal(signal.SIGPIPE, signal.SIG_DFL) # keep default SIGPIPE behavior

    I = None
    frame_count = 0
    fps_start_time = time.time()
    # Wait for the first frame before entering the main loop
    #if not reucam.frame_event.wait(5): # Wait up to 5 seconds for the first frame
    #    logging.error("Failed to receive first frame within timeout. Exiting.")
    #    sys.exit(1)
   
    while True:
        if reucam.frame_event.wait(1):  # check every 1 second
            break
        logging.info("Waiting for camera to connect...")

    #for i in range(1,100, 10):
    #    param1, param2 = convert_bboxes_to_reucam_params([100+i, 50+i, 300+i, 200+i])
    #    reucam.onFireBorder(param1, param2)
    #    time.sleep(1)
    #    reucam.onFireBorder(0, 0)

    track_fail_time = None  # time of first track failure
    tracker = cv2.TrackerCSRT_create()
    while True:
        line = check_stdin()
        if line == "RESET_BORDER":
            logging.info("Received RESET_BORDER command")
            reucam.onFireBorder(0, 0)
            mode = 1
            bbox_mode3 = None
            flicker_stable_count = 0
            roi_stable_count = 0
            fourier_buff[:] = 0
   
                               
        reucam.frame_event.clear()
        #reucam.onFireBorder(param1, param2)
        # If current frame equals to previous frame or its None - continue
        if reucam.I == I:
            #logging.debug("Frame(I) is equal to previous Frame(I), skipping.")
            continue
        I = reucam.I
        ts = reucam.ts

        frame_count += 1
        current_time = time.time()
        if current_time - fps_start_time >= 1.0:  # Every second
            fps = frame_count / (current_time - fps_start_time)
            logging.info(f"FPS: {fps:.2f}")
            frame_count = 0
            fps_start_time = current_time
            FPS=fps
            #print('fps',fps)
       
        if I is None:
            logging.warning("Frame is of type None, skipping.")
            continue


        yuv_data = reucam.I

        # ===== MODE 1 OPTIMIZATION: Use Y channel directly for frequency detection =====
        if mode == 1:
            # For Mode 1, extract Y channel directly from YUV data to avoid expensive color conversions
            y_size = FRAME_W * FRAME_H
            gray = np.frombuffer(yuv_data[0:y_size], dtype=np.uint8).reshape((FRAME_H, FRAME_W))
            _, B2 = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
            # Create a simple BGR frame for visualization (Y channel only)
            frame = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
            frame[10:20,260:380,:]=0 
        else:
            # For Modes 2 & 3, use full color conversion pipeline for yellow LED detection
            frame = convert_yuv_to_bgr(yuv_data, FRAME_W, FRAME_H)
            frame[10:20,260:380,:]=0 
            
            #! =============== HSV COLOR FILTER CODE ============================================================
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

            lower_bound = np.array([31, 10, 184])
            upper_bound = np.array([66, 255, 255])

            mask = cv2.inRange(hsv, lower_bound, upper_bound)
            filtered = cv2.bitwise_and(frame, frame, mask=mask)
            #! =============== HSV COLOR FILTER CODE ============================================================
            
            #! =============== GREEN CHANNEL [:,:,1] AND thresh=200 ============================================================
            gray = filtered[:, :, 1]  # Green channel (vs blue channel [:,:,2] in Cell 1)
            _, B2 = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)  # thresh=200 (vs 250 in Cell 1)
            # !=============== GREEN CHANNEL [:,:,1] AND thresh=200 ============================================================
        # ===== END MODE 1 OPTIMIZATION =====


        if mode == 1:
            
            #tracker = cv2.legacy.TrackerCSRT_create() if hasattr(cv2, 'legacy') else cv2.TrackerMIL_create()
            #tracker = cv2.legacy.TrackerCSRT_create()

            logging.info("mode==1")
            if track_fail_time is not None:
                logging.info(f"current_time - track_fail_time = {current_time - track_fail_time}")
            current_time = time.time()
            if track_fail_time is not None and current_time - track_fail_time >= 5.0:
                print("1 0 0", flush=True)
                logging.info("mode 1: 1 0 0")
            elif track_fail_time is None:
                print("1 0 0", flush=True)
                logging.info("mode 1: 1 0 0")
            #print("1 0 0", flush=True)
           
           
            found_boxes = []
            for h in range(HE):
                for w in range(WI):
                    cv2.rectangle(frame,
                                  (w*BOX_SIZE, h*BOX_SIZE),
                                  (w*BOX_SIZE+BOX_SIZE, h*BOX_SIZE+BOX_SIZE),
                                  (255, 255, 255), 1)

                    box, freq = process_box(frame, gray, B2, h, w)

                    #cv2.putText(frame, str(freq), (w*BOX_SIZE, h*BOX_SIZE), cv2.FONT_HERSHEY_SIMPLEX,0.6, (0,0,255), 2)
                    if box is not None:
                        found_boxes.append(box)

            if found_boxes:
                x1 = max(0, min(b[0] for b in found_boxes) - MARGIN)
                y1 = max(0, min(b[1] for b in found_boxes) - MARGIN)
                x2 = min(frame_width,  max(b[2] for b in found_boxes) + MARGIN)
                y2 = min(frame_height, max(b[3] for b in found_boxes) + MARGIN)

                cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,255), 2)

                roi_stable_count += 1
                if roi_stable_count >= ROI_STABLE_FRAMES:
                    bbox_mode2 = (x1 - MARGIN, y1 - MARGIN, x2 + MARGIN, y2 + MARGIN)
                    top_point = (x1- MARGIN, y1- MARGIN)
                    bottom_point = (x2+ MARGIN, y2+ MARGIN)
                    mode = 2
            else:
                roi_stable_count = 0
   
               
        elif mode == 2:
            logging.info("mode==2")  
            current_time = time.time()
            if track_fail_time is not None:
                logging.info(f"current_time - track_fail_time = {current_time - track_fail_time}")
            if track_fail_time is not None and current_time - track_fail_time >= 5.0:
                print("1 0 0", flush=True)
                logging.info("mode 2: 1 0 0")
            elif track_fail_time is None:
                print("1 0 0", flush=True)
                logging.info("mode 2: 1 0 0")
            #print("1 0 0", flush=True)

            if bbox_mode2 is not None:
                x1, y1, x2, y2 = bbox_mode2
                cv2.rectangle(frame, (x1,y1), (x2,y2), (255, 0, 255), 2)
            mask, flicker_buff, frame_idx,bbox_mode3 = process_flicker_pixels(frame, bbox_mode2, flicker_buff, frame_idx)
            if bbox_mode3:
                (x0, y0, w0, h0) = [int(v) for v in bbox_mode3]

        elif mode == 3:
            cross_conf = configparser.ConfigParser()
            cross_conf.read(cross_conf_path)
            center_x0 = cross_conf.getint("camera", "cross_w_pos", fallback=frame_width // 2)
            center_y0 = cross_conf.getint("camera", "cross_h_pos", fallback=frame_height // 2)
            logging.info("mode==3")
            if bbox_mode3 is not None:
                try:
                    logging.info(f"Frame shape: {frame.shape}, bbox_mode3: {bbox_mode3}")
                except Exception as e:
                    logging.error(f"Frame info error: {e}")
                   
                x, y, w, h = bbox_mode3
                if x < 0 or y < 0 or x + w > frame.shape[1] or y + h > frame.shape[0]:
                    logging.error(f"Invalid bbox before tracker.update(): {bbox_mode3}")
                    success = False
                    continue

                try:
                    logging.info("tracker.update()")
                    success, bbox = tracker.update(frame)
                    #tracker.init(frame, bbox)
                except Exception as e:
                    logging.exception(f"tracker.update() failed: {e}")
                    success = False

                #success, bbox = tracker.update(frame)
                logging.info("if bbox_mode3 is not None:")
                if success:    
                    track_fail_time = None    # reset the fail timer because tracking succeeded
                    logging.info(f"Frame shape: {frame.shape}, bbox: {bbox}")
                    #tracker.init(frame, bbox)
                    x, y, w, h = [int(v) for v in bbox]
                    x = max(0, x)
                    y = max(0, y)
                    w= min(frame.shape[1]-1-x, w)
                    h = min(frame.shape[0]-y-1, h)
                    bbox_debug=(x,y,w,h)
                    
                    #! =============== DISTANCE CALCULATION AND SAFETY CHECK ============================================================
                    Distance=(LED_CIRCLE_SIZE/(2*w*np.tan(np.deg2rad(0.5*IFOV_x))))/MARGIN3_factor
                    print('Distance', Distance)
                    if Distance<2:
                        reucam.onFireBorder(0, 0)
                        mode = 1
                        tcounter = 0
                        failure_count_m3 = 0
                        pixel_count_buff[:] = 0
                        flicker_stable_count = 0
                        bbox_mode3 = None        
                        flicker_buff = None      
                        roi_stable_count = 0    
                        fourier_buff = np.zeros((CALC_LENGTH, HE, WI))
                    #! ===============  DISTANCE CALCULATION AND SAFETY CHECK ============================================================ 
                                       
                    #tracker.init(frame, bbox_debug)
                    logging.info(f"Frame shape: {frame.shape}, bbox_d: {bbox}")
                    
                    center_x = (x + w // 2)
                    center_y = (y + h // 2)
                    bbox_draw=[x,y,x+w,y+h]
                    param1, param2 = convert_bboxes_to_reucam_params(bbox_draw)
                   
                    # inside mode 3 loop
                    param1, param2 = convert_bboxes_to_reucam_params(bbox_draw)
                    current_time = time.time()
                    if last_fireborder != (param1, param2) or current_time - last_fire_time > 0.05:
                        reucam.onFireBorder(0,0)
                        time.sleep(0.01)
                        reucam.onFireBorder(param1, param2)
                        last_fireborder = (param1, param2)
                        last_fire_time = current_time
                   
                   
                    #reucam.onFireBorder(0, 0)
                    #time.sleep(0.01)
                    #reucam.onFireBorder(param1, param2)
                    #
                    #reucam.onFireBorder(0, 0)
                   
                    #center_text = f"Center: ({center_x}, {center_y})"

                    x_alignment=center_x-center_x0
                    y_alignment = center_y - center_y0
                    ANG_THRESH=0.2
                    y_alignment_deg=-2*np.round((center_x-center_x0)*IFOV_x,1)      # transmit to Israel with a negative sign
                    y_alignment_deg = 0 if abs(y_alignment_deg) < ANG_THRESH else y_alignment_deg
                    x_alignment_deg = -2*np.round((center_y - center_y0)*IFOV_y,1)  # transmit to Israel with a negative sign
                    x_alignment_deg = 0 if abs(x_alignment_deg) < ANG_THRESH else x_alignment_deg
                   
                    logging.info(f"x={x_alignment_deg} y={y_alignment_deg}")
                    try:
                        print(f"0 {x_alignment_deg} {y_alignment_deg}", flush=True)
                    except BrokenPipeError:
                        # just ignore the error and continue running
                        sys.stderr.write("stdout closed, ignoring...\n")

                    if (np.abs(x_alignment_deg)>0.5) or (np.abs(y_alignment_deg)>0.5):
                        color=(0,0,255)
                    else:
                        color=(255,0,0)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2, 1)
                    center_text2= f"({x_alignment_deg}, {y_alignment_deg})"
                    center_text = f"({x_alignment}, {y_alignment})"
                    center_text3= f"({center_x}, {center_y})"
                    text_position = (x, y + h + 20)
                    text2_position = (x, y + h + 40)
                    cv2.putText(frame, center_text2, text_position, cv2.FONT_HERSHEY_SIMPLEX,0.6, color, 2)
                    #cv2.putText(frame, center_text2, text2_position, cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

                    roi_binary = B2[y:y+h, x:x+w]
                    non_zero_pixel_count = np.sum(roi_binary)
                    pixel_count_buff = np.roll(pixel_count_buff, -1)
                    pixel_count_buff[-1] = non_zero_pixel_count
                    #print('tcounter',tcounter)
                    tcounter += 1
                    if tcounter > CALC_LENGTH:
                        detected_freq = get_fft_frequency(pixel_count_buff, FPS)

                        if abs(detected_freq - LED_FREQ) > FREQ_TOL:

                            failure_count_m3 += 1
                            #print('counter',str(failure_count_m3))
                            if failure_count_m3 >= MAX_FAILURES_M3:
                                logging.info('Freq failed')
                                reucam.onFireBorder(0, 0)
                                mode = 1
                                tcounter = 0
                                failure_count_m3 = 0
                                pixel_count_buff[:] = 0
                                flicker_stable_count = 0
                                bbox_mode3 = None        
                                flicker_buff = None      
                                roi_stable_count = 0    
                                fourier_buff = np.zeros((CALC_LENGTH, HE, WI))
                               

                        else:
                            failure_count_m3 = 0
                else:
                    logging.info('Track failed')
                    roi_stable_count = 0
                    reucam.onFireBorder(0, 0)
                    mode = 1
                    flicker_stable_count = 0
                    bbox_mode3 = None
                    failure_count_m3 = 0
                    roi_stable_count = 0
                    fourier_buff = np.zeros((CALC_LENGTH, HE, WI))
                    #tracker = cv2.legacy.TrackerKCF_create()
                   
                    current_time = time.time()
                    if track_fail_time is None:
                        # first failure, start the 5-second timer
                        track_fail_time = current_time
                        logging.info('Track failed - starting 5 second wait')
                    elif current_time - track_fail_time >= 5.0:
                        logging.info('else mode 3: Track failed')
                        print("1 0 0", flush=True)

            else:
                print("1 0 0", flush=True)
       
        else:
            print("1 0 0", flush=True)
   
        cv2.putText(frame, f"Mode: {mode}", (40, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)

           
        
        
        #cv2.imshow("preview", cv2.resize(mask, (640, 480)))
        key = cv2.waitKey(1) & 0xFF
        if key == 27 or key == ord('q'):
            break

In [ ]:
# my changes
#!/usr/bin/python3

# Simple ReuVision Viewer - fixed
import struct
import sys
import threading
import logging
import time
import cv2
import numpy as np
from scipy.signal import find_peaks
import configparser
import os
import json
import math
import time
import signal
import atexit
import fcntl

cv2.ocl.setUseOpenCL(False)
cv2.setNumThreads(1)

signal.signal(signal.SIGPIPE, signal.SIG_DFL)    # Exit immediately when stdout is closed (SIGPIPE)

sys.path.append("/var/fgvideo/src")
sys.path.append("/var/fgvideo/src/dist-packages")

from ReuCameraPy.ReuCameraPy import *
from ReuOnvifPy import config

logging.basicConfig(level=logging.INFO, format="[%(levelname).1s] %(message)s")

FRAME_W, FRAME_H = config.get_snapshot_resolution()


# Load JSON config
config_path = os.path.join(os.path.dirname(__file__), 'camera_alignment_config.json')
try:
    with open(config_path, 'r') as f:
        cfg = json.load(f)
except FileNotFoundError:
    logging.error(f"Configuration file not found at {config_path}")
    sys.exit(1)
except json.JSONDecodeError:
    logging.error(f"Error decoding JSON from {config_path}")
    sys.exit(1)

REUCAM_NOTIFY_FIREBORDER = 0x8010
sensor_width = cfg["sensor_width"]
sensor_height = cfg["sensor_height"]
pixel_size = cfg["pixel_size"]
focal_length = cfg["focal_length"]
#FPS = cfg["FPS"]
BOX_SIZE = cfg["BOX_SIZE"]
LED_FREQ = cfg["LED_FREQ"]
FREQ_TOL = cfg["FREQ_TOL"]
CALC_LENGTH = cfg["CALC_LENGTH"]
MARGIN = cfg["MARGIN"]
fourier_peak_threshold = cfg["fourier_peak_threshold"]


frame_width = FRAME_W
frame_height = FRAME_H

pixel_size_eff_x = pixel_size * sensor_width / frame_width
pixel_size_eff_y = pixel_size * sensor_height / frame_height

IFOV_x = 2 * np.degrees(np.arctan(pixel_size_eff_x / (2 * focal_length)))
IFOV_y = 2 * np.degrees(np.arctan(pixel_size_eff_y / (2 * focal_length)))

# ===== Global buffers =====
WI, HE = frame_width // BOX_SIZE+1, frame_height // BOX_SIZE+1
fourier_buff = np.zeros((CALC_LENGTH, HE, WI))
freq_counter = np.zeros((HE, WI))
pixel_count_buff = np.zeros(CALC_LENGTH)


# ===== Global variables =====
mode = 1    # Initial mode - general scan
roi_mode1 = None  # Will be defined in mode 1
MARGIN3_factor= 1
LED_CIRCLE_SIZE=0.05
bbox_mode2 = None # Search area for mode 2
flicker_buff = None
scounter = tcounter = m2counter = 0
top_point, bottom_point = None, None
pixel_buffer = None
frame_idx = 0
# Variables for transition to mode 3
flicker_stable_count = 0
STABLE_FRAMES_THRESHOLD = 5  # Number of frames the contour must remain to transition to mode 3
bbox_mode3 = None  # The rectangle that is transitioned to mode 3
ROI_STABLE_FRAMES = 1
ROI_HISTORY = 10    # How many frames to keep (e.g. 10 = about a third of a second at 30fps)
roi_history = []    # List of detected block masks
failure_count_m3=0
MAX_FAILURES_M3=40
roi_stable_count = 0
frame_idx = 0
flicker_buff = None


# ===== Read FPS from /etc/reucamd.conf =====
fps_conf = configparser.ConfigParser()
fps_conf.read("/etc/reucamd.conf")

FPS = fps_conf.getint("camera", "framerate", fallback=cfg["FPS"])

# ===== Read cross position from dev/fgvideo/etc/reucamd.conf =====
cross_conf_path = os.path.join(os.path.dirname(__file__), '..', 'etc', 'reucamd.conf')




PIXEL_HISTORY = FPS * 2


# -----------------------------
# Reucam Service
# -----------------------------
class ReucamClient(ReuCameraClient):
    def __init__(self, svc):
        super().__init__()
        self.svc = svc

    def cmd_onFrame(self, cmd):
        frame = cmd.frame
        enc = frame.enc
        self.svc.onFrame(frame)

class ReucamService(threading.Thread):
    def __init__(self):
        super().__init__()
        self.daemon = True
        self.client = None
        self.lock = threading.Lock()
        self.I = None
        self.ts = None
        self.frame_event = threading.Event()
        self.bActive = False

    def run(self):
        self.bActive = True
        logging.info("ReucamService: started")
        backoff = 1.0
        while self.bActive:
            client = None
            try:
                client = ReucamClient(self)
                self.client = client
                client.connect(ReuCameraService.SERVER_ADDR)
                logging.info("ReucamService: connected")
                client.cmd_StartStream(3)  # RAW

                # recv_loop may raise when peer closes; catch inside loop via exception below
                client.recv_loop()

            except Exception:
                # log full traceback (important for debugging 'Peer closed' etc.)
                logging.exception("ReucamService error - reconnecting after failure")
                # ensure we don't keep a broken client reference
                try:
                    if client:
                        client.close()
                except Exception:
                    logging.exception("Error while closing client after failure")
                self.client = None

                # exponential backoff with cap
                time.sleep(backoff)
                backoff = min(backoff * 2, 30.0)
                continue
            else:
                # clean shutdown of recv_loop (no exception) -> reset backoff
                backoff = 1.0
                try:
                    if client:
                        client.close()
                except Exception:
                    logging.exception("Error while closing client in normal shutdown")
                self.client = None
            finally:
                # small delay to avoid tight loop if bActive still True
                time.sleep(0.2)

    def onFrame(self, frame):
        # frame may be None or malformed; guard it
        try:
            if not frame:
                return
            if getattr(frame, 'is_raw', False) and len(frame.data) == FRAME_W * FRAME_H * 3 // 2:
                with self.lock:
                    self.I = frame.data
                    self.ts = frame.timestamp
                    self.frame_event.set()
            else:
                # log unexpected frames at debug level
                logging.debug("Received non-raw or unexpected-size frame")
        except Exception:
            logging.exception("Error in onFrame handler")

    def onFireBorder(self, param1, param2):
        # protect against missing/closed client
        try:
            if self.client is None:
                logging.debug("onFireBorder called but client is None (not connected)")
                return
            # do not call into a client that might be closed; wrap in try
            try:
                self.client.cmd_Notify(REUCAM_NOTIFY_FIREBORDER, param1, param2)
            except Exception:
                logging.exception("Failed to send Notify to reucam client")
        except Exception:
            logging.exception("Unexpected error in onFireBorder")


# -----------------------------
# Auxiliary functions
# -----------------------------
#   ======================================================================================================================================================
def convert_yuv_to_bgr(yuv_data, width, height):
    y_size = width * height
    uv_size = y_size // 4
    y = np.frombuffer(yuv_data[0:y_size], dtype=np.uint8).reshape((height, width))
    u = np.frombuffer(yuv_data[y_size:y_size+uv_size], dtype=np.uint8).reshape((height//2, width//2))
    v = np.frombuffer(yuv_data[y_size+uv_size:y_size+2*uv_size], dtype=np.uint8).reshape((height//2, width//2))
    u_up = cv2.resize(u, (width, height), interpolation=cv2.INTER_LINEAR)
    v_up = cv2.resize(v, (width, height), interpolation=cv2.INTER_LINEAR)
    yuv = cv2.merge((y, u_up, v_up))
    bgr = cv2.cvtColor(yuv, cv2.COLOR_YUV2BGR)
    return bgr
#def convert_yuv_to_bgr(yuv_data, width, height):
    #"""
    #Converts a YUV420 frame (byte array) to a BGR image (OpenCV format).
    #"""
    #if yuv_data is None:
    #    return None

    #frame_gray = np.frombuffer(yuv_data[0:width * height], dtype=np.uint8).reshape((height, width))
    #frame = cv2.cvtColor(frame_gray, cv2.COLOR_GRAY2BGR)

    #return frame
#   ======================================================================================================================================================
def get_fft_frequency(signal, fps):
    signal = signal - np.mean(signal)
    fft_vals = np.abs(np.fft.rfft(signal))[1:]
    f_axis = np.linspace(0, fps / 2, len(fft_vals) + 1)[1:]
    if len(fft_vals) == 0:
        return 0
    peaks, _ = find_peaks(fft_vals, height=np.max(fft_vals))
    #if len(peaks):
    #    print(f_axis[peaks[0]])
    if len(peaks) > 0 and fft_vals[peaks[0]] > fourier_peak_threshold * np.median(fft_vals):
        return f_axis[peaks[0]]
    return 0

def process_box(frame, gray, B2, h, w):
    b2 = B2[h*BOX_SIZE:(h+1)*BOX_SIZE, w*BOX_SIZE:(w+1)*BOX_SIZE]

    # ????? ????????? FFT ?????
    z = np.roll(fourier_buff[:, h, w], 1)      # ???? np.roll ?? ndimage.shift
    z[0] = np.sum(b2)
    fourier_buff[:, h, w] = z

    # FFT
    freq = get_fft_frequency(fourier_buff[:, h, w], FPS)
    #print(freq)
    # ????? ?????? (?-ROI ?? ??????)
    if abs(freq - LED_FREQ) < FREQ_TOL:
        # ?????????: ????? ???? ????? ??? freq_counter ??????? ???????
        freq_counter[h, w] += 1
        x1 = w * BOX_SIZE
        y1 = h * BOX_SIZE
        x2 = min((w + 1) * BOX_SIZE, frame_width)
        y2 = min((h + 1) * BOX_SIZE, frame_height)
        return (x1, y1, x2, y2), freq

    # ?? ??? ????? � ??? ?????? ????? ??
    freq_counter[h, w] = 0
    return None, None
    
    


#! =============== DIFFERENCE: CELL 2 HAS thresh=100 (vs 140 in Cell 1) ===============
def process_flicker_pixels(frame, roi, flicker_buff, frame_count, buffer_len=10, thresh=100):
    global flicker_stable_count, mode, bbox_mode3
    if roi is None:
        return None, flicker_buff, frame_count, bbox_mode3
#! =============== DIFFERENCE: CELL 2 HAS thresh=100 (vs 140 in Cell 1) ===============

    x1, y1, x2, y2 = roi
    x1 = max(0, x1)
    y1 = max(0, y1)
    x2 = min(frame.shape[1]-1, x2)
    y2 = min(frame.shape[0]-1, y2)

    if x2 <= x1 or y2 <= y1:
        return None, flicker_buff, frame_count, bbox_mode3
    roi_gray = cv2.cvtColor(frame[y1:y2, x1:x2], cv2.COLOR_BGR2GRAY)
    h, w = roi_gray.shape

    if flicker_buff is None or flicker_buff.shape[1:] != (h, w):
        flicker_buff = np.zeros((buffer_len, h, w), dtype=np.uint8)
        frame_count = 0
        flicker_stable_count = 0

    flicker_buff = np.roll(flicker_buff, -1, axis=0)
    flicker_buff[-1] = roi_gray
    frame_count += 1

    mask = np.zeros((h, w), dtype=np.uint8)
    if frame_count > buffer_len:
        diff = flicker_buff.max(axis=0) - flicker_buff.min(axis=0)
        mask[diff > thresh] = 200
        #! =============== DIFFERENCE: CELL 2 USES DYNAMIC MARGIN3 CALCULATION ===============
        #MARGIN3=25
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            all_points = np.vstack(contours)
            x, y, w_box, h_box = cv2.boundingRect(all_points)

            MARGIN3 = int(MARGIN3_factor * max(w_box, h_box))  # Dynamic calculation
            x_margin = max(0, x1 + x - MARGIN3)
            y_margin = max(0, y1 + y - MARGIN3)
            x2_margin = min(frame.shape[1], x1 + x + w_box + MARGIN3)
            y2_margin = min(frame.shape[0], y1 + y + h_box + MARGIN3)
            cv2.rectangle(frame, (x1+x, y1+y), (x1+x+w_box, y1+y+h_box), (0, 0, 255), 2)
        #! =============== DIFFERENCE: CELL 2 USES DYNAMIC MARGIN3 CALCULATION ===============
            flicker_stable_count += 1
            
                       
            if flicker_stable_count >= STABLE_FRAMES_THRESHOLD:
               
               
                #bbox_mode3 = (x1+x, y1+y, w_box, h_box)
                bbox_mode3 = (x_margin, y_margin,x2_margin - x_margin, y2_margin - y_margin)
                tracker.init(frame, bbox_mode3)
                logging.info("tracker.init(frame, bbox_mode3)")
                logging.info(f"bbox_mode3={bbox_mode3}")
                mode = 3
        else:
            flicker_stable_count = 0
            bbox_mode3 = None
            mode = 1

    return mask, flicker_buff, frame_count, bbox_mode3

def convert_bboxes_to_reucam_params(bbox):
    """
    Converts a bounding box to ReuCam parameters safely.
    bbox = [x0, y0, x1, y1]
    Returns param1, param2 (packed)
    """

    # Scale coordinates to FRAME_W / FRAME_H
    x0 = math.ceil(bbox[0] * FRAME_W / frame_width)
    x1 = math.ceil(bbox[2] * FRAME_W / frame_width)
    y0 = math.ceil(bbox[1] * FRAME_H / frame_height)
    y1 = math.ceil(bbox[3] * FRAME_H / frame_height)

    w = x1 - x0
    h = y1 - y0

    # If frame is large, reduce w/h
    if frame_width > 640:
        w = math.ceil(w / 2)
        h = math.ceil(h / 2)

    # Clamp values to valid ranges
    x0 = max(0, min(65535, x0))
    y0 = max(0, min(65535, y0))  # ushort range
    w = max(0, min(255, w))      # B range
    h = max(0, min(255, h))      # B range

    param1 = x0
    try:
        param2 = struct.pack('HBB', y0, w, h)
    except struct.error as e:
        logging.error(f"struct.pack failed: y0={y0}, w={w}, h={h}")
        param2 = struct.pack('HBB', 0, 0, 0)

    param2 = struct.unpack('i', param2)[0]

    return param1, param2

# Set stdin to non-blocking once at startup
fd = sys.stdin.fileno()
fl = fcntl.fcntl(fd, fcntl.F_GETFL)
fcntl.fcntl(fd, fcntl.F_SETFL, fl | os.O_NONBLOCK)

def check_stdin():
    try:
        line = sys.stdin.readline()
        if line:
            return line.strip()
    except Exception:
        return None
    return None

last_fireborder = None
last_fire_time = 0

# -----------------------------
# Main
# -----------------------------
if __name__ == "__main__":
    reucam = ReucamService()
    reucam.start()
    def cleanup():
        try:
            reucam.onFireBorder(0, 0)
        except Exception:
            pass  # ignore errors if reucam is not available yet
   
    # Register cleanup for normal exit
    atexit.register(cleanup)
   
    # Handle termination signals
    def handle_signal(signum, frame):
        cleanup()
        sys.exit(0)
   
    signal.signal(signal.SIGINT, handle_signal)   # CTRL+C
    signal.signal(signal.SIGTERM, handle_signal)  # kill command
    signal.signal(signal.SIGPIPE, signal.SIG_DFL) # keep default SIGPIPE behavior

    I = None
    frame_count = 0
    fps_start_time = time.time()
    # Wait for the first frame before entering the main loop
    #if not reucam.frame_event.wait(5): # Wait up to 5 seconds for the first frame
    #    logging.error("Failed to receive first frame within timeout. Exiting.")
    #    sys.exit(1)
   
    while True:
        if reucam.frame_event.wait(1):  # check every 1 second
            break
        logging.info("Waiting for camera to connect...")

    #for i in range(1,100, 10):
    #    param1, param2 = convert_bboxes_to_reucam_params([100+i, 50+i, 300+i, 200+i])
    #    reucam.onFireBorder(param1, param2)
    #    time.sleep(1)
    #    reucam.onFireBorder(0, 0)

    track_fail_time = None  # time of first track failure
    tracker = cv2.TrackerCSRT_create()
    while True:
        line = check_stdin()
        if line == "RESET_BORDER":
            logging.info("Received RESET_BORDER command")
            reucam.onFireBorder(0, 0)
            mode = 1
            bbox_mode3 = None
            flicker_stable_count = 0
            roi_stable_count = 0
            fourier_buff[:] = 0
   
                               
        reucam.frame_event.clear()
        #reucam.onFireBorder(param1, param2)
        # If current frame equals to previous frame or its None - continue
        if reucam.I == I:
            #logging.debug("Frame(I) is equal to previous Frame(I), skipping.")
            continue
        I = reucam.I
        ts = reucam.ts

        frame_count += 1
        current_time = time.time()
        if current_time - fps_start_time >= 1.0:  # Every second
            fps = frame_count / (current_time - fps_start_time)
            logging.info(f"FPS: {fps:.2f}")
            frame_count = 0
            fps_start_time = current_time
            FPS=fps
            #print('fps',fps)
       
        if I is None:
            logging.warning("Frame is of type None, skipping.")
            continue


        yuv_data = reucam.I

        # ===== MODE 1 OPTIMIZATION: Use Y channel directly for frequency detection =====
        if mode == 1:
            # For Mode 1, extract Y channel directly from YUV data to avoid expensive color conversions
            y_size = FRAME_W * FRAME_H
            gray = np.frombuffer(yuv_data[0:y_size], dtype=np.uint8).reshape((FRAME_H, FRAME_W))
            _, B2 = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
            # Create a simple BGR frame for visualization (Y channel only)
            frame = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
            frame[10:20,260:380,:]=0 
        else:
            # For Modes 2 & 3, use full color conversion pipeline for yellow LED detection
            frame = convert_yuv_to_bgr(yuv_data, FRAME_W, FRAME_H)
            frame[10:20,260:380,:]=0 
            
            #! =============== HSV COLOR FILTER CODE ============================================================
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

            lower_bound = np.array([31, 10, 184])
            upper_bound = np.array([66, 255, 255])

            mask = cv2.inRange(hsv, lower_bound, upper_bound)
            filtered = cv2.bitwise_and(frame, frame, mask=mask)
            #! =============== HSV COLOR FILTER CODE ============================================================
            
            #! =============== GREEN CHANNEL [:,:,1] AND thresh=200 ============================================================
            gray = filtered[:, :, 1]  # Green channel (vs blue channel [:,:,2] in Cell 1)
            _, B2 = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)  # thresh=200 (vs 250 in Cell 1)
            # !=============== GREEN CHANNEL [:,:,1] AND thresh=200 ============================================================
        # ===== END MODE 1 OPTIMIZATION =====


        if mode == 1:
            
            #tracker = cv2.legacy.TrackerCSRT_create() if hasattr(cv2, 'legacy') else cv2.TrackerMIL_create()
            #tracker = cv2.legacy.TrackerCSRT_create()

            # ===== PERFORMANCE OPTIMIZATION: Reduce excessive logging =====
            # logging.info("mode==1")  # Commented out - too frequent
            # if track_fail_time is not None:
            #     logging.info(f"current_time - track_fail_time = {current_time - track_fail_time}")  # Commented out - too frequent
            current_time = time.time()
            if track_fail_time is not None and current_time - track_fail_time >= 5.0:
                print("1 0 0", flush=True)
                # logging.info("mode 1: 1 0 0")  # Commented out - too frequent
            elif track_fail_time is None:
                print("1 0 0", flush=True)
                # logging.info("mode 1: 1 0 0")  # Commented out - too frequent
            #print("1 0 0", flush=True)
           
           
            found_boxes = []
            for h in range(HE):
                for w in range(WI):
                    # ===== PERFORMANCE OPTIMIZATION: Reduce grid drawing overhead =====
                    # Only draw grid every 10th frame to reduce cv2.rectangle calls
                    if frame_count % 10 == 0:
                        cv2.rectangle(frame,
                                      (w*BOX_SIZE, h*BOX_SIZE),
                                      (w*BOX_SIZE+BOX_SIZE, h*BOX_SIZE+BOX_SIZE),
                                      (255, 255, 255), 1)

                    box, freq = process_box(frame, gray, B2, h, w)

                    #cv2.putText(frame, str(freq), (w*BOX_SIZE, h*BOX_SIZE), cv2.FONT_HERSHEY_SIMPLEX,0.6, (0,0,255), 2)
                    if box is not None:
                        found_boxes.append(box)

            if found_boxes:
                x1 = max(0, min(b[0] for b in found_boxes) - MARGIN)
                y1 = max(0, min(b[1] for b in found_boxes) - MARGIN)
                x2 = min(frame_width,  max(b[2] for b in found_boxes) + MARGIN)
                y2 = min(frame_height, max(b[3] for b in found_boxes) + MARGIN)

                cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,255), 2)

                roi_stable_count += 1
                if roi_stable_count >= ROI_STABLE_FRAMES:
                    bbox_mode2 = (x1 - MARGIN, y1 - MARGIN, x2 + MARGIN, y2 + MARGIN)
                    top_point = (x1- MARGIN, y1- MARGIN)
                    bottom_point = (x2+ MARGIN, y2+ MARGIN)
                    mode = 2
            else:
                roi_stable_count = 0
   
               
        elif mode == 2:
            logging.info("mode==2")  
            current_time = time.time()
            if track_fail_time is not None:
                logging.info(f"current_time - track_fail_time = {current_time - track_fail_time}")
            if track_fail_time is not None and current_time - track_fail_time >= 5.0:
                print("1 0 0", flush=True)
                logging.info("mode 2: 1 0 0")
            elif track_fail_time is None:
                print("1 0 0", flush=True)
                logging.info("mode 2: 1 0 0")
            #print("1 0 0", flush=True)

            if bbox_mode2 is not None:
                x1, y1, x2, y2 = bbox_mode2
                cv2.rectangle(frame, (x1,y1), (x2,y2), (255, 0, 255), 2)
            mask, flicker_buff, frame_idx,bbox_mode3 = process_flicker_pixels(frame, bbox_mode2, flicker_buff, frame_idx)
            if bbox_mode3:
                (x0, y0, w0, h0) = [int(v) for v in bbox_mode3]

        elif mode == 3:
            cross_conf = configparser.ConfigParser()
            cross_conf.read(cross_conf_path)
            center_x0 = cross_conf.getint("camera", "cross_w_pos", fallback=frame_width // 2)
            center_y0 = cross_conf.getint("camera", "cross_h_pos", fallback=frame_height // 2)
            logging.info("mode==3")
            if bbox_mode3 is not None:
                try:
                    logging.info(f"Frame shape: {frame.shape}, bbox_mode3: {bbox_mode3}")
                except Exception as e:
                    logging.error(f"Frame info error: {e}")
                   
                x, y, w, h = bbox_mode3
                if x < 0 or y < 0 or x + w > frame.shape[1] or y + h > frame.shape[0]:
                    logging.error(f"Invalid bbox before tracker.update(): {bbox_mode3}")
                    success = False
                    continue

                try:
                    logging.info("tracker.update()")
                    success, bbox = tracker.update(frame)
                    #tracker.init(frame, bbox)
                except Exception as e:
                    logging.exception(f"tracker.update() failed: {e}")
                    success = False

                #success, bbox = tracker.update(frame)
                logging.info("if bbox_mode3 is not None:")
                if success:    
                    track_fail_time = None    # reset the fail timer because tracking succeeded
                    logging.info(f"Frame shape: {frame.shape}, bbox: {bbox}")
                    #tracker.init(frame, bbox)
                    x, y, w, h = [int(v) for v in bbox]
                    x = max(0, x)
                    y = max(0, y)
                    w= min(frame.shape[1]-1-x, w)
                    h = min(frame.shape[0]-y-1, h)
                    bbox_debug=(x,y,w,h)
                    
                    #! =============== DISTANCE CALCULATION AND SAFETY CHECK ============================================================
                    Distance=(LED_CIRCLE_SIZE/(2*w*np.tan(np.deg2rad(0.5*IFOV_x))))/MARGIN3_factor
                    print('Distance', Distance)
                    if Distance<2:
                        reucam.onFireBorder(0, 0)
                        mode = 1
                        tcounter = 0
                        failure_count_m3 = 0
                        pixel_count_buff[:] = 0
                        flicker_stable_count = 0
                        bbox_mode3 = None        
                        flicker_buff = None      
                        roi_stable_count = 0    
                        fourier_buff = np.zeros((CALC_LENGTH, HE, WI))
                    #! ===============  DISTANCE CALCULATION AND SAFETY CHECK ============================================================ 
                                       
                    #tracker.init(frame, bbox_debug)
                    logging.info(f"Frame shape: {frame.shape}, bbox_d: {bbox}")
                    
                    center_x = (x + w // 2)
                    center_y = (y + h // 2)
                    bbox_draw=[x,y,x+w,y+h]
                    param1, param2 = convert_bboxes_to_reucam_params(bbox_draw)
                   
                    # inside mode 3 loop
                    param1, param2 = convert_bboxes_to_reucam_params(bbox_draw)
                    current_time = time.time()
                    if last_fireborder != (param1, param2) or current_time - last_fire_time > 0.05:
                        reucam.onFireBorder(0,0)
                        time.sleep(0.01)
                        reucam.onFireBorder(param1, param2)
                        last_fireborder = (param1, param2)
                        last_fire_time = current_time
                   
                   
                    #reucam.onFireBorder(0, 0)
                    #time.sleep(0.01)
                    #reucam.onFireBorder(param1, param2)
                    #
                    #reucam.onFireBorder(0, 0)
                   
                    #center_text = f"Center: ({center_x}, {center_y})"

                    x_alignment=center_x-center_x0
                    y_alignment = center_y - center_y0
                    ANG_THRESH=0.2
                    y_alignment_deg=-2*np.round((center_x-center_x0)*IFOV_x,1)      # transmit to Israel with a negative sign
                    y_alignment_deg = 0 if abs(y_alignment_deg) < ANG_THRESH else y_alignment_deg
                    x_alignment_deg = -2*np.round((center_y - center_y0)*IFOV_y,1)  # transmit to Israel with a negative sign
                    x_alignment_deg = 0 if abs(x_alignment_deg) < ANG_THRESH else x_alignment_deg
                   
                    logging.info(f"x={x_alignment_deg} y={y_alignment_deg}")
                    try:
                        print(f"0 {x_alignment_deg} {y_alignment_deg}", flush=True)
                    except BrokenPipeError:
                        # just ignore the error and continue running
                        sys.stderr.write("stdout closed, ignoring...\n")

                    if (np.abs(x_alignment_deg)>0.5) or (np.abs(y_alignment_deg)>0.5):
                        color=(0,0,255)
                    else:
                        color=(255,0,0)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2, 1)
                    center_text2= f"({x_alignment_deg}, {y_alignment_deg})"
                    center_text = f"({x_alignment}, {y_alignment})"
                    center_text3= f"({center_x}, {center_y})"
                    text_position = (x, y + h + 20)
                    text2_position = (x, y + h + 40)
                    cv2.putText(frame, center_text2, text_position, cv2.FONT_HERSHEY_SIMPLEX,0.6, color, 2)
                    #cv2.putText(frame, center_text2, text2_position, cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

                    roi_binary = B2[y:y+h, x:x+w]
                    non_zero_pixel_count = np.sum(roi_binary)
                    pixel_count_buff = np.roll(pixel_count_buff, -1)
                    pixel_count_buff[-1] = non_zero_pixel_count
                    #print('tcounter',tcounter)
                    tcounter += 1
                    if tcounter > CALC_LENGTH:
                        detected_freq = get_fft_frequency(pixel_count_buff, FPS)

                        if abs(detected_freq - LED_FREQ) > FREQ_TOL:

                            failure_count_m3 += 1
                            #print('counter',str(failure_count_m3))
                            if failure_count_m3 >= MAX_FAILURES_M3:
                                logging.info('Freq failed')
                                reucam.onFireBorder(0, 0)
                                mode = 1
                                tcounter = 0
                                failure_count_m3 = 0
                                pixel_count_buff[:] = 0
                                flicker_stable_count = 0
                                bbox_mode3 = None        
                                flicker_buff = None      
                                roi_stable_count = 0    
                                fourier_buff = np.zeros((CALC_LENGTH, HE, WI))
                               

                        else:
                            failure_count_m3 = 0
                else:
                    logging.info('Track failed')
                    roi_stable_count = 0
                    reucam.onFireBorder(0, 0)
                    mode = 1
                    flicker_stable_count = 0
                    bbox_mode3 = None
                    failure_count_m3 = 0
                    roi_stable_count = 0
                    fourier_buff = np.zeros((CALC_LENGTH, HE, WI))
                    #tracker = cv2.legacy.TrackerKCF_create()
                   
                    current_time = time.time()
                    if track_fail_time is None:
                        # first failure, start the 5-second timer
                        track_fail_time = current_time
                        logging.info('Track failed - starting 5 second wait')
                    elif current_time - track_fail_time >= 5.0:
                        logging.info('else mode 3: Track failed')
                        print("1 0 0", flush=True)

            else:
                print("1 0 0", flush=True)
       
        else:
            print("1 0 0", flush=True)
   
        cv2.putText(frame, f"Mode: {mode}", (40, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)

           
        
        
        #cv2.imshow("preview", cv2.resize(mask, (640, 480)))
        key = cv2.waitKey(1) & 0xFF
        if key == 27 or key == ord('q'):
            break

In [ ]:
# MODE 3 OPTIMIZED VERSION - Major Performance Improvements
#!/usr/bin/python3

# Simple ReuVision Viewer - fixed
import struct
import sys
import threading
import logging
import time
import cv2
import numpy as np
from scipy.signal import find_peaks
import configparser
import os
import json
import math
import time
import signal
import atexit
import fcntl

cv2.ocl.setUseOpenCL(False)
cv2.setNumThreads(1)

signal.signal(signal.SIGPIPE, signal.SIG_DFL)    # Exit immediately when stdout is closed (SIGPIPE)

sys.path.append("/var/fgvideo/src")
sys.path.append("/var/fgvideo/src/dist-packages")

from ReuCameraPy.ReuCameraPy import *
from ReuOnvifPy import config

logging.basicConfig(level=logging.INFO, format="[%(levelname).1s] %(message)s")

FRAME_W, FRAME_H = config.get_snapshot_resolution()

# Load JSON config
config_path = os.path.join(os.path.dirname(__file__), 'camera_alignment_config.json')
try:
    with open(config_path, 'r') as f:
        cfg = json.load(f)
except FileNotFoundError:
    logging.error(f"Configuration file not found at {config_path}")
    sys.exit(1)
except json.JSONDecodeError:
    logging.error(f"Error decoding JSON from {config_path}")
    sys.exit(1)

REUCAM_NOTIFY_FIREBORDER = 0x8010
sensor_width = cfg["sensor_width"]
sensor_height = cfg["sensor_height"]
pixel_size = cfg["pixel_size"]
focal_length = cfg["focal_length"]
BOX_SIZE = cfg["BOX_SIZE"]
LED_FREQ = cfg["LED_FREQ"]
FREQ_TOL = cfg["FREQ_TOL"]
CALC_LENGTH = cfg["CALC_LENGTH"]
MARGIN = cfg["MARGIN"]
fourier_peak_threshold = cfg["fourier_peak_threshold"]

frame_width = FRAME_W
frame_height = FRAME_H

pixel_size_eff_x = pixel_size * sensor_width / frame_width
pixel_size_eff_y = pixel_size * sensor_height / frame_height

IFOV_x = 2 * np.degrees(np.arctan(pixel_size_eff_x / (2 * focal_length)))
IFOV_y = 2 * np.degrees(np.arctan(pixel_size_eff_y / (2 * focal_length)))

# ===== Global buffers =====
WI, HE = frame_width // BOX_SIZE+1, frame_height // BOX_SIZE+1
fourier_buff = np.zeros((CALC_LENGTH, HE, WI))
freq_counter = np.zeros((HE, WI))
pixel_count_buff = np.zeros(CALC_LENGTH)

# ===== Global variables =====
mode = 1    # Initial mode - general scan
roi_mode1 = None  # Will be defined in mode 1
MARGIN3_factor= 1
LED_CIRCLE_SIZE=0.05
bbox_mode2 = None # Search area for mode 2
flicker_buff = None
scounter = tcounter = m2counter = 0
top_point, bottom_point = None, None
pixel_buffer = None
frame_idx = 0
# Variables for transition to mode 3
flicker_stable_count = 0
STABLE_FRAMES_THRESHOLD = 5  # Number of frames the contour must remain to transition to mode 3
bbox_mode3 = None  # The rectangle that is transitioned to mode 3
ROI_STABLE_FRAMES = 1
ROI_HISTORY = 10    # How many frames to keep (e.g. 10 = about a third of a second at 30fps)
roi_history = []    # List of detected block masks
failure_count_m3=0
MAX_FAILURES_M3=40
roi_stable_count = 0
frame_idx = 0
flicker_buff = None

# ===== OPTIMIZATION: Pre-load cross configuration to avoid file reads every frame =====
cross_conf_path = os.path.join(os.path.dirname(__file__), '..', 'etc', 'reucamd.conf')
cross_conf = configparser.ConfigParser()
cross_conf.read(cross_conf_path)
center_x0_cached = cross_conf.getint("camera", "cross_w_pos", fallback=frame_width // 2)
center_y0_cached = cross_conf.getint("camera", "cross_h_pos", fallback=frame_height // 2)

# ===== Read FPS from /etc/reucamd.conf =====
fps_conf = configparser.ConfigParser()
fps_conf.read("/etc/reucamd.conf")
FPS = fps_conf.getint("camera", "framerate", fallback=cfg["FPS"])

PIXEL_HISTORY = FPS * 2

# -----------------------------
# Reucam Service
# -----------------------------
class ReucamClient(ReuCameraClient):
    def __init__(self, svc):
        super().__init__()
        self.svc = svc

    def cmd_onFrame(self, cmd):
        frame = cmd.frame
        enc = frame.enc
        self.svc.onFrame(frame)

class ReucamService(threading.Thread):
    def __init__(self):
        super().__init__()
        self.daemon = True
        self.client = None
        self.lock = threading.Lock()
        self.I = None
        self.ts = None
        self.frame_event = threading.Event()
        self.bActive = False

    def run(self):
        self.bActive = True
        logging.info("ReucamService: started")
        backoff = 1.0
        while self.bActive:
            client = None
            try:
                client = ReucamClient(self)
                self.client = client
                client.connect(ReuCameraService.SERVER_ADDR)
                logging.info("ReucamService: connected")
                client.cmd_StartStream(3)  # RAW

                # recv_loop may raise when peer closes; catch inside loop via exception below
                client.recv_loop()

            except Exception:
                # log full traceback (important for debugging 'Peer closed' etc.)
                logging.exception("ReucamService error - reconnecting after failure")
                # ensure we don't keep a broken client reference
                try:
                    if client:
                        client.close()
                except Exception:
                    logging.exception("Error while closing client after failure")
                self.client = None

                # exponential backoff with cap
                time.sleep(backoff)
                backoff = min(backoff * 2, 30.0)
                continue
            else:
                # clean shutdown of recv_loop (no exception) -> reset backoff
                backoff = 1.0
                try:
                    if client:
                        client.close()
                except Exception:
                    logging.exception("Error while closing client in normal shutdown")
                self.client = None
            finally:
                # small delay to avoid tight loop if bActive still True
                time.sleep(0.2)

    def onFrame(self, frame):
        # frame may be None or malformed; guard it
        try:
            if not frame:
                return
            if getattr(frame, 'is_raw', False) and len(frame.data) == FRAME_W * FRAME_H * 3 // 2:
                with self.lock:
                    self.I = frame.data
                    self.ts = frame.timestamp
                    self.frame_event.set()
            else:
                # log unexpected frames at debug level
                logging.debug("Received non-raw or unexpected-size frame")
        except Exception:
            logging.exception("Error in onFrame handler")

    def onFireBorder(self, param1, param2):
        # protect against missing/closed client
        try:
            if self.client is None:
                logging.debug("onFireBorder called but client is None (not connected)")
                return
            # do not call into a client that might be closed; wrap in try
            try:
                self.client.cmd_Notify(REUCAM_NOTIFY_FIREBORDER, param1, param2)
            except Exception:
                logging.exception("Failed to send Notify to reucam client")
        except Exception:
            logging.exception("Unexpected error in onFireBorder")

# -----------------------------
# Auxiliary functions
# -----------------------------
def convert_yuv_to_bgr(yuv_data, width, height):
    y_size = width * height
    uv_size = y_size // 4
    y = np.frombuffer(yuv_data[0:y_size], dtype=np.uint8).reshape((height, width))
    u = np.frombuffer(yuv_data[y_size:y_size+uv_size], dtype=np.uint8).reshape((height//2, width//2))
    v = np.frombuffer(yuv_data[y_size+uv_size:y_size+2*uv_size], dtype=np.uint8).reshape((height//2, width//2))
    u_up = cv2.resize(u, (width, height), interpolation=cv2.INTER_LINEAR)
    v_up = cv2.resize(v, (width, height), interpolation=cv2.INTER_LINEAR)
    yuv = cv2.merge((y, u_up, v_up))
    bgr = cv2.cvtColor(yuv, cv2.COLOR_YUV2BGR)
    return bgr

def get_fft_frequency(signal, fps):
    signal = signal - np.mean(signal)
    fft_vals = np.abs(np.fft.rfft(signal))[1:]
    f_axis = np.linspace(0, fps / 2, len(fft_vals) + 1)[1:]
    if len(fft_vals) == 0:
        return 0
    peaks, _ = find_peaks(fft_vals, height=np.max(fft_vals))
    if len(peaks) > 0 and fft_vals[peaks[0]] > fourier_peak_threshold * np.median(fft_vals):
        return f_axis[peaks[0]]
    return 0

def process_box(frame, gray, B2, h, w):
    b2 = B2[h*BOX_SIZE:(h+1)*BOX_SIZE, w*BOX_SIZE:(w+1)*BOX_SIZE]
    z = np.roll(fourier_buff[:, h, w], 1)
    z[0] = np.sum(b2)
    fourier_buff[:, h, w] = z
    freq = get_fft_frequency(fourier_buff[:, h, w], FPS)
    if abs(freq - LED_FREQ) < FREQ_TOL:
        freq_counter[h, w] += 1
        x1 = w * BOX_SIZE
        y1 = h * BOX_SIZE
        x2 = min((w + 1) * BOX_SIZE, frame_width)
        y2 = min((h + 1) * BOX_SIZE, frame_height)
        return (x1, y1, x2, y2), freq
    freq_counter[h, w] = 0
    return None, None

def process_flicker_pixels(frame, roi, flicker_buff, frame_count, buffer_len=10, thresh=100):
    global flicker_stable_count, mode, bbox_mode3
    if roi is None:
        return None, flicker_buff, frame_count, bbox_mode3

    x1, y1, x2, y2 = roi
    x1 = max(0, x1)
    y1 = max(0, y1)
    x2 = min(frame.shape[1]-1, x2)
    y2 = min(frame.shape[0]-1, y2)

    if x2 <= x1 or y2 <= y1:
        return None, flicker_buff, frame_count, bbox_mode3
    roi_gray = cv2.cvtColor(frame[y1:y2, x1:x2], cv2.COLOR_BGR2GRAY)
    h, w = roi_gray.shape

    if flicker_buff is None or flicker_buff.shape[1:] != (h, w):
        flicker_buff = np.zeros((buffer_len, h, w), dtype=np.uint8)
        frame_count = 0
        flicker_stable_count = 0

    flicker_buff = np.roll(flicker_buff, -1, axis=0)
    flicker_buff[-1] = roi_gray
    frame_count += 1

    mask = np.zeros((h, w), dtype=np.uint8)
    if frame_count > buffer_len:
        diff = flicker_buff.max(axis=0) - flicker_buff.min(axis=0)
        mask[diff > thresh] = 200
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            all_points = np.vstack(contours)
            x, y, w_box, h_box = cv2.boundingRect(all_points)

            MARGIN3 = int(MARGIN3_factor * max(w_box, h_box))  # Dynamic calculation
            x_margin = max(0, x1 + x - MARGIN3)
            y_margin = max(0, y1 + y - MARGIN3)
            x2_margin = min(frame.shape[1], x1 + x + w_box + MARGIN3)
            y2_margin = min(frame.shape[0], y1 + y + h_box + MARGIN3)
            cv2.rectangle(frame, (x1+x, y1+y), (x1+x+w_box, y1+y+h_box), (0, 0, 255), 2)
            flicker_stable_count += 1
            
            if flicker_stable_count >= STABLE_FRAMES_THRESHOLD:
                bbox_mode3 = (x_margin, y_margin,x2_margin - x_margin, y2_margin - y_margin)
                tracker.init(frame, bbox_mode3)
                logging.info("tracker.init(frame, bbox_mode3)")
                logging.info(f"bbox_mode3={bbox_mode3}")
                mode = 3
        else:
            flicker_stable_count = 0
            bbox_mode3 = None
            mode = 1

    return mask, flicker_buff, frame_count, bbox_mode3

def convert_bboxes_to_reucam_params(bbox):
    """
    Converts a bounding box to ReuCam parameters safely.
    bbox = [x0, y0, x1, y1]
    Returns param1, param2 (packed)
    """
    # Scale coordinates to FRAME_W / FRAME_H
    x0 = math.ceil(bbox[0] * FRAME_W / frame_width)
    x1 = math.ceil(bbox[2] * FRAME_W / frame_width)
    y0 = math.ceil(bbox[1] * FRAME_H / frame_height)
    y1 = math.ceil(bbox[3] * FRAME_H / frame_height)

    w = x1 - x0
    h = y1 - y0

    # If frame is large, reduce w/h
    if frame_width > 640:
        w = math.ceil(w / 2)
        h = math.ceil(h / 2)

    # Clamp values to valid ranges
    x0 = max(0, min(65535, x0))
    y0 = max(0, min(65535, y0))  # ushort range
    w = max(0, min(255, w))      # B range
    h = max(0, min(255, h))      # B range

    param1 = x0
    try:
        param2 = struct.pack('HBB', y0, w, h)
    except struct.error as e:
        logging.error(f"struct.pack failed: y0={y0}, w={w}, h={h}")
        param2 = struct.pack('HBB', 0, 0, 0)

    param2 = struct.unpack('i', param2)[0]

    return param1, param2

# Set stdin to non-blocking once at startup
fd = sys.stdin.fileno()
fl = fcntl.fcntl(fd, fcntl.F_GETFL)
fcntl.fcntl(fd, fcntl.F_SETFL, fl | os.O_NONBLOCK)

def check_stdin():
    try:
        line = sys.stdin.readline()
        if line:
            return line.strip()
    except Exception:
        return None
    return None

last_fireborder = None
last_fire_time = 0

# -----------------------------
# Main
# -----------------------------
if __name__ == "__main__":
    reucam = ReucamService()
    reucam.start()
    def cleanup():
        try:
            reucam.onFireBorder(0, 0)
        except Exception:
            pass  # ignore errors if reucam is not available yet
   
    # Register cleanup for normal exit
    atexit.register(cleanup)
   
    # Handle termination signals
    def handle_signal(signum, frame):
        cleanup()
        sys.exit(0)
   
    signal.signal(signal.SIGINT, handle_signal)   # CTRL+C
    signal.signal(signal.SIGTERM, handle_signal)  # kill command
    signal.signal(signal.SIGPIPE, signal.SIG_DFL) # keep default SIGPIPE behavior

    I = None
    frame_count = 0
    fps_start_time = time.time()
   
    while True:
        if reucam.frame_event.wait(1):  # check every 1 second
            break
        logging.info("Waiting for camera to connect...")

    track_fail_time = None  # time of first track failure
    tracker = cv2.TrackerCSRT_create()
    
    # ===== MODE 3 OPTIMIZATION: Pre-calculate constants =====
    ANG_THRESH = 0.2
    
    while True:
        line = check_stdin()
        if line == "RESET_BORDER":
            logging.info("Received RESET_BORDER command")
            reucam.onFireBorder(0, 0)
            mode = 1
            bbox_mode3 = None
            flicker_stable_count = 0
            roi_stable_count = 0
            fourier_buff[:] = 0
   
        reucam.frame_event.clear()
        
        # If current frame equals to previous frame or its None - continue
        if reucam.I == I:
            continue
        I = reucam.I
        ts = reucam.ts

        frame_count += 1
        current_time = time.time()
        
        # ===== OPTIMIZATION: Reduce FPS logging frequency =====
        if current_time - fps_start_time >= 3.0:  # Every 3 seconds instead of every second
            fps = frame_count / (current_time - fps_start_time)
            logging.info(f"FPS: {fps:.2f}")
            frame_count = 0
            fps_start_time = current_time
            FPS = fps
       
        if I is None:
            logging.warning("Frame is of type None, skipping.")
            continue

        yuv_data = reucam.I

        # ===== MODE 1 OPTIMIZATION: Use Y channel directly for frequency detection =====
        if mode == 1:
            # For Mode 1, extract Y channel directly from YUV data to avoid expensive color conversions
            y_size = FRAME_W * FRAME_H
            gray = np.frombuffer(yuv_data[0:y_size], dtype=np.uint8).reshape((FRAME_H, FRAME_W))
            _, B2 = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
            # Create a simple BGR frame for visualization (Y channel only)
            frame = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
            frame[10:20,260:380,:]=0 
        else:
            # For Modes 2 & 3, use full color conversion pipeline for yellow LED detection
            frame = convert_yuv_to_bgr(yuv_data, FRAME_W, FRAME_H)
            frame[10:20,260:380,:]=0 
            
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            lower_bound = np.array([31, 10, 184])
            upper_bound = np.array([66, 255, 255])
            mask = cv2.inRange(hsv, lower_bound, upper_bound)
            filtered = cv2.bitwise_and(frame, frame, mask=mask)
            
            gray = filtered[:, :, 1]  # Green channel
            _, B2 = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)

        if mode == 1:
            # ===== MODE 1 OPTIMIZATIONS: Reduced logging and grid drawing =====
            current_time = time.time()
            if track_fail_time is not None and current_time - track_fail_time >= 5.0:
                print("1 0 0", flush=True)
            elif track_fail_time is None:
                print("1 0 0", flush=True)
           
            found_boxes = []
            for h in range(HE):
                for w in range(WI):
                    # Only draw grid every 10th frame to reduce cv2.rectangle calls
                    if frame_count % 10 == 0:
                        cv2.rectangle(frame,
                                      (w*BOX_SIZE, h*BOX_SIZE),
                                      (w*BOX_SIZE+BOX_SIZE, h*BOX_SIZE+BOX_SIZE),
                                      (255, 255, 255), 1)

                    box, freq = process_box(frame, gray, B2, h, w)
                    if box is not None:
                        found_boxes.append(box)

            if found_boxes:
                x1 = max(0, min(b[0] for b in found_boxes) - MARGIN)
                y1 = max(0, min(b[1] for b in found_boxes) - MARGIN)
                x2 = min(frame_width,  max(b[2] for b in found_boxes) + MARGIN)
                y2 = min(frame_height, max(b[3] for b in found_boxes) + MARGIN)

                cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,255), 2)

                roi_stable_count += 1
                if roi_stable_count >= ROI_STABLE_FRAMES:
                    bbox_mode2 = (x1 - MARGIN, y1 - MARGIN, x2 + MARGIN, y2 + MARGIN)
                    top_point = (x1- MARGIN, y1- MARGIN)
                    bottom_point = (x2+ MARGIN, y2+ MARGIN)
                    mode = 2
            else:
                roi_stable_count = 0
   
        elif mode == 2:
            # ===== MODE 2 OPTIMIZATIONS: Reduced logging =====
            current_time = time.time()
            if track_fail_time is not None and current_time - track_fail_time >= 5.0:
                print("1 0 0", flush=True)
            elif track_fail_time is None:
                print("1 0 0", flush=True)

            if bbox_mode2 is not None:
                x1, y1, x2, y2 = bbox_mode2
                cv2.rectangle(frame, (x1,y1), (x2,y2), (255, 0, 255), 2)
            mask, flicker_buff, frame_idx,bbox_mode3 = process_flicker_pixels(frame, bbox_mode2, flicker_buff, frame_idx)
            if bbox_mode3:
                (x0, y0, w0, h0) = [int(v) for v in bbox_mode3]

        elif mode == 3:
            # ===== MODE 3 MAJOR OPTIMIZATIONS =====
            # Use cached cross position instead of reading config file every frame
            center_x0 = center_x0_cached
            center_y0 = center_y0_cached
            
            if bbox_mode3 is not None:
                x, y, w, h = bbox_mode3
                if x < 0 or y < 0 or x + w > frame.shape[1] or y + h > frame.shape[0]:
                    logging.error(f"Invalid bbox before tracker.update(): {bbox_mode3}")
                    success = False
                    continue

                try:
                    success, bbox = tracker.update(frame)
                except Exception as e:
                    logging.exception(f"tracker.update() failed: {e}")
                    success = False

                if success:    
                    track_fail_time = None    # reset the fail timer because tracking succeeded
                    x, y, w, h = [int(v) for v in bbox]
                    x = max(0, x)
                    y = max(0, y)
                    w = min(frame.shape[1]-1-x, w)
                    h = min(frame.shape[0]-y-1, h)
                    
                    # ===== OPTIMIZATION: Safety check with minimal logging =====
                    Distance = (LED_CIRCLE_SIZE/(2*w*np.tan(np.deg2rad(0.5*IFOV_x))))/MARGIN3_factor
                    if Distance < 2:
                        reucam.onFireBorder(0, 0)
                        mode = 1
                        tcounter = 0
                        failure_count_m3 = 0
                        pixel_count_buff[:] = 0
                        flicker_stable_count = 0
                        bbox_mode3 = None        
                        flicker_buff = None      
                        roi_stable_count = 0    
                        fourier_buff = np.zeros((CALC_LENGTH, HE, WI))
                        continue
                    
                    # ===== OPTIMIZATION: Calculate alignment efficiently =====
                    center_x = x + w // 2
                    center_y = y + h // 2
                    bbox_draw = [x, y, x+w, y+h]
                    param1, param2 = convert_bboxes_to_reucam_params(bbox_draw)
                   
                    # ===== OPTIMIZATION: Reduce fireborder calls =====
                    current_time = time.time()
                    if last_fireborder != (param1, param2) or current_time - last_fire_time > 0.05:
                        reucam.onFireBorder(0,0)
                        time.sleep(0.01)
                        reucam.onFireBorder(param1, param2)
                        last_fireborder = (param1, param2)
                        last_fire_time = current_time

                    # ===== OPTIMIZATION: Pre-calculated angular alignment =====
                    x_alignment = center_x - center_x0
                    y_alignment = center_y - center_y0
                    y_alignment_deg = -2*np.round((center_x-center_x0)*IFOV_x, 1)
                    y_alignment_deg = 0 if abs(y_alignment_deg) < ANG_THRESH else y_alignment_deg
                    x_alignment_deg = -2*np.round((center_y - center_y0)*IFOV_y, 1)
                    x_alignment_deg = 0 if abs(x_alignment_deg) < ANG_THRESH else x_alignment_deg
                   
                    try:
                        print(f"0 {x_alignment_deg} {y_alignment_deg}", flush=True)
                    except BrokenPipeError:
                        sys.stderr.write("stdout closed, ignoring...\n")

                    # ===== OPTIMIZATION: Simplified visualization =====
                    color = (0,0,255) if (np.abs(x_alignment_deg)>0.5) or (np.abs(y_alignment_deg)>0.5) else (255,0,0)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    center_text = f"({x_alignment_deg}, {y_alignment_deg})"
                    text_position = (x, y + h + 20)
                    cv2.putText(frame, center_text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

                    # ===== OPTIMIZATION: Frequency detection =====
                    roi_binary = B2[y:y+h, x:x+w]
                    non_zero_pixel_count = np.sum(roi_binary)
                    pixel_count_buff = np.roll(pixel_count_buff, -1)
                    pixel_count_buff[-1] = non_zero_pixel_count
                    tcounter += 1
                    
                    if tcounter > CALC_LENGTH:
                        detected_freq = get_fft_frequency(pixel_count_buff, FPS)
                        if abs(detected_freq - LED_FREQ) > FREQ_TOL:
                            failure_count_m3 += 1
                            if failure_count_m3 >= MAX_FAILURES_M3:
                                # logging.info('Freq failed')  # Commented out for performance
                                reucam.onFireBorder(0, 0)
                                mode = 1
                                tcounter = 0
                                failure_count_m3 = 0
                                pixel_count_buff[:] = 0
                                flicker_stable_count = 0
                                bbox_mode3 = None        
                                flicker_buff = None      
                                roi_stable_count = 0    
                                fourier_buff = np.zeros((CALC_LENGTH, HE, WI))
                        else:
                            failure_count_m3 = 0
                            
                else:
                    # ===== OPTIMIZATION: Simplified track failure handling =====
                    roi_stable_count = 0
                    reucam.onFireBorder(0, 0)
                    mode = 1
                    flicker_stable_count = 0
                    bbox_mode3 = None
                    failure_count_m3 = 0
                    roi_stable_count = 0
                    fourier_buff = np.zeros((CALC_LENGTH, HE, WI))
                   
                    current_time = time.time()
                    if track_fail_time is None:
                        # first failure, start the 5-second timer
                        track_fail_time = current_time
                        # logging.info('Track failed - starting 5 second wait')  # Commented out for performance
                    elif current_time - track_fail_time >= 5.0:
                        print("1 0 0", flush=True)
            else:
                print("1 0 0", flush=True)
       
        else:
            print("1 0 0", flush=True)
   
        cv2.putText(frame, f"Mode: {mode}", (40, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)

        key = cv2.waitKey(1) & 0xFF
        if key == 27 or key == ord('q'):
            break

In [ ]:
# NEW:

# Video Recording Cell - Add RAW video recording capability (CLEAN VERSION)
# !/usr/bin/python3

# Simple ReuVision Viewer - with RAW Video Recording
import struct
import sys
import threading
import logging
import time
import cv2
import numpy as np
from scipy.signal import find_peaks
import configparser
import os
import json
import math
import time
import signal
import atexit
import fcntl

cv2.ocl.setUseOpenCL(False)
cv2.setNumThreads(1)

signal.signal(signal.SIGPIPE, signal.SIG_DFL)    # Exit immediately when stdout is closed (SIGPIPE)

sys.path.append("/var/fgvideo/src")
sys.path.append("/var/fgvideo/src/dist-packages")

from ReuCameraPy.ReuCameraPy import *
from ReuOnvifPy import config

logging.basicConfig(level=logging.INFO, format="[%(levelname).1s] %(message)s")

FRAME_W, FRAME_H = config.get_snapshot_resolution()

# Load JSON config
config_path = os.path.join(os.path.dirname(__file__), 'camera_alignment_config.json')
try:
    with open(config_path, 'r') as f:
        cfg = json.load(f)
except FileNotFoundError:
    logging.error(f"Configuration file not found at {config_path}")
    sys.exit(1)
except json.JSONDecodeError:
    logging.error(f"Error decoding JSON from {config_path}")
    sys.exit(1)

# ===== GLOBAL PARAMETERS CONFIGURATION =====
# All hardcoded values are centralized here for easy tuning

# Camera and sensor parameters (from config file)
REUCAM_NOTIFY_FIREBORDER = 0x8010  # ReuCam notification ID for fire border detection
sensor_width = cfg["sensor_width"]
sensor_height = cfg["sensor_height"]
pixel_size = cfg["pixel_size"]
focal_length = cfg["focal_length"]
BOX_SIZE = cfg["BOX_SIZE"]
LED_FREQ = cfg["LED_FREQ"]
FREQ_TOL = cfg["FREQ_TOL"]
CALC_LENGTH = cfg["CALC_LENGTH"]
MARGIN = cfg["MARGIN"]
fourier_peak_threshold = cfg["fourier_peak_threshold"]

# Video recording parameters
VIDEO_CODEC = 'mp4v'  # Video codec for recording - 'mp4v' for compatibility, 'MJPG' for quality
RECORDING_STATUS_UPDATE_INTERVAL = 60  # Update recording status every N frames (~2 seconds at 30fps)

# Mode 1: Frequency detection parameters
BINARY_THRESHOLD_MODE1 = 200  # Binary threshold for LED detection in Mode 1 (0-255, higher = only bright pixels)
GRID_COLOR_MODE1 = (255, 255, 255)  # RGB color for Mode 1 detection grid overlay (white)
GRID_LINE_THICKNESS = 1  # Line thickness for Mode 1 grid visuali zation (pixels)
ROI_STABLE_FRAMES_MODE1 = 1  # Number of consecutive frames ROI must be stable before transitioning to Mode 2

# Mode 2: HSV color filtering parameters  
HSV_LOWER_BOUND = np.array([21, 10, 184])  # Lower HSV bound for yellow LED detection (Hue, Saturation, Value)
HSV_UPPER_BOUND = np.array([80, 255, 240])  # Upper HSV bound for yellow LED detection
FLICKER_BUFFER_LENGTH = 10  # Number of frames to store for flicker analysis (~0.33s at 30fps)
FLICKER_DIFF_THRESHOLD = 150  # Minimum pixel intensity difference to detect flickering (0-255)
STABLE_FRAMES_THRESHOLD_MODE2 = 5  # Frames contour must remain stable to transition to Mode 3
CONTOUR_BOX_COLOR = (0, 0, 255)  # RGB color for Mode 2 contour bounding box (red)
CONTOUR_BOX_THICKNESS = 2  # Line thickness for Mode 2 contour box

# Mode 3: Object tracking parameters
TRACKER_TIMEOUT_SECONDS = 5.0  # Time to wait after tracking failure before resetting to Mode 1
MAX_TRACKING_FAILURES = 40  # Maximum consecutive frequency validation failures before reset
ALIGNMENT_ANGLE_THRESHOLD = 0.2  # Minimum angle difference to report alignment (degrees)
ALIGNMENT_DEADBAND = 0.5  # Minimum alignment error to highlight in red (degrees)
TRACKING_BOX_COLOR = (0, 255, 0)  # RGB color for Mode 3 tracking box (green)
TRACKING_BOX_THICKNESS = 2  # Line thickness for Mode 3 tracking box
ALIGNMENT_TEXT_SCALE = 0.6  # Font scale for alignment text display
CROSS_HAIR_UPDATE_INTERVAL = 0.05  # Minimum time between cross-hair updates (seconds)

# Distance calculation parameters  
LED_CIRCLE_SIZE_FACTOR = 0.05  # Physical LED circle size factor for distance calculation
MARGIN3_SAFETY_FACTOR = 25  # Fixed margin for Mode 3 bounding box expansion (pixels)
DISTANCE_SAFETY_CHECK = 0  # Minimum valid distance threshold (negative values trigger reset)

# Frame processing parameters
FRAME_TIMEOUT_SECONDS = 1.0  # Timeout for waiting for new frames (seconds)
CLOCK_REGION_Y1, CLOCK_REGION_Y2 = 10, 20  # Y coordinates for clock region removal
CLOCK_REGION_X1, CLOCK_REGION_X2 = 260, 380  # X coordinates for clock region removal
FPS_CALCULATION_INTERVAL = 1.0  # Time interval for FPS calculation updates (seconds)

# Display and UI parameters
MODE_TEXT_POSITION = (40, 80)  # Position for mode display text
MODE_TEXT_SCALE = 1.5  # Font scale for mode display
MODE_TEXT_COLOR = (255, 255, 255)  # RGB color for mode text (white)
MODE_TEXT_THICKNESS = 2  # Thickness for mode text
RECORDING_TEXT_POSITION = (10, 30)  # Position for recording status text
RECORDING_TEXT_SCALE = 0.7  # Font scale for recording status
RECORDING_TEXT_COLOR = (0, 0, 255)  # RGB color for recording text (red)
RECORDING_TEXT_THICKNESS = 2  # Thickness for recording status text

# System and performance parameters
ROI_HISTORY_LENGTH = 10  # Number of ROI detection frames to keep in history (~0.33s at 30fps)
PIXEL_HISTORY_MULTIPLIER = 2  # Multiplier for pixel history buffer size (FPS * multiplier)

# ===== END GLOBAL PARAMETERS =====

frame_width = FRAME_W
frame_height = FRAME_H

pixel_size_eff_x = pixel_size * sensor_width / frame_width
pixel_size_eff_y = pixel_size * sensor_height / frame_height

IFOV_x = 2 * np.degrees(np.arctan(pixel_size_eff_x / (2 * focal_length)))
IFOV_y = 2 * np.degrees(np.arctan(pixel_size_eff_y / (2 * focal_length)))

# ===== Global buffers =====
WI, HE = frame_width // BOX_SIZE+1, frame_height // BOX_SIZE+1
fourier_buff = np.zeros((CALC_LENGTH, HE, WI))
freq_counter = np.zeros((HE, WI))
pixel_count_buff = np.zeros(CALC_LENGTH)

# ===== Global variables =====
mode = 1    # Initial mode - general scan
roi_mode1 = None  # Will be defined in mode 1
MARGIN3_factor= 1
LED_CIRCLE_SIZE=LED_CIRCLE_SIZE_FACTOR
bbox_mode2 = None # Search area for mode 2
flicker_buff = None
scounter = tcounter = m2counter = 0
top_point, bottom_point = None, None
pixel_buffer = None
frame_idx = 0
# Variables for transition to mode 3
flicker_stable_count = 0
bbox_mode3 = None  # The rectangle that is transitioned to mode 3
ROI_STABLE_FRAMES = ROI_STABLE_FRAMES_MODE1
ROI_HISTORY = ROI_HISTORY_LENGTH    # How many frames to keep (e.g. 10 = about a third of a second at 30fps)
roi_history = []    # List of detected block masks
failure_count_m3=0
MAX_FAILURES_M3=MAX_TRACKING_FAILURES
roi_stable_count = 0
frame_idx = 0
flicker_buff = None

# ===== VIDEO RECORDING VARIABLES =====
is_recording = False
video_writer = None
recording_filename = ""
frame_count_record = 0
recording_start_time = None

# ===== Read FPS from /etc/reucamd.conf =====
fps_conf = configparser.ConfigParser()
fps_conf.read("/etc/reucamd.conf")

FPS = fps_conf.getint("camera", "framerate", fallback=cfg["FPS"])

# ===== Read cross position from dev/fgvideo/etc/reucamd.conf =====
cross_conf_path = os.path.join(os.path.dirname(__file__), '..', 'etc', 'reucamd.conf')

PIXEL_HISTORY = FPS * PIXEL_HISTORY_MULTIPLIER

# -----------------------------
# VIDEO RECORDING FUNCTIONS
# -----------------------------
def start_recording():
    """Start RAW video recording"""
    global is_recording, video_writer, recording_filename, frame_count_record, recording_start_time
    
    if is_recording:
        logging.info("Already recording!")
        return
    
    # Generate filename with timestamp for RAW video
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    recording_filename = f"RAW_camera_recording_{timestamp}.mp4"
    
    # Define codec and create VideoWriter for RAW recording
    fourcc = cv2.VideoWriter_fourcc(*VIDEO_CODEC)  # Use global codec parameter
    video_writer = cv2.VideoWriter(
        recording_filename, 
        fourcc, 
        FPS,  # Use actual camera FPS
        (FRAME_W, FRAME_H)
    )
    
    if not video_writer.isOpened():
        logging.error("Failed to initialize RAW video writer")
        return
    
    is_recording = True
    recording_start_time = time.time()
    frame_count_record = 0
    
    logging.info(f"RAW Recording started: {recording_filename}")
    print(f"RAW_RECORDING_STARTED: {recording_filename}")

def stop_recording():
    """Stop RAW video recording"""
    global is_recording, video_writer, recording_filename, frame_count_record
    
    if not is_recording:
        logging.info("Not recording!")
        return
    
    if video_writer:
        video_writer.release()
        video_writer = None
    
    recording_duration = time.time() - recording_start_time if recording_start_time else 0
    
    is_recording = False
    
    logging.info(f"RAW Recording stopped. Saved: {recording_filename} ({frame_count_record} frames, {recording_duration:.1f}s)")
    print(f"RAW_RECORDING_STOPPED: {recording_filename}, frames: {frame_count_record}, duration: {recording_duration:.1f}s")

def record_raw_frame(yuv_data):
    """Record RAW YUV frame to video file"""
    global frame_count_record
    
    if is_recording and video_writer and yuv_data is not None:
        # Convert YUV to BGR for recording (RAW, no overlays)
        raw_frame = convert_yuv_to_bgr(yuv_data, FRAME_W, FRAME_H)
        
        # Write RAW frame (no boxes, no overlays, no processing)
        video_writer.write(raw_frame)
        frame_count_record += 1
        
        # Update recording status every N frames using global parameter
        if frame_count_record % RECORDING_STATUS_UPDATE_INTERVAL == 0:
            elapsed_time = time.time() - recording_start_time
            logging.info(f"Recording... {frame_count_record} frames ({elapsed_time:.1f}s)")

# -----------------------------
# Reucam Service
# -----------------------------
class ReucamClient(ReuCameraClient):
    def __init__(self, svc):
        super().__init__()
        self.svc = svc

    def cmd_onFrame(self, cmd):
        frame = cmd.frame
        enc = frame.enc
        self.svc.onFrame(frame)

class ReucamService(threading.Thread):
    def __init__(self):
        super().__init__()
        self.daemon = True
        self.client = None
        self.lock = threading.Lock()
        self.I = None
        self.ts = None
        self.frame_event = threading.Event()
        self.bActive = False

    def run(self):
        self.bActive = True
        logging.info("ReucamService: started")
        backoff = 1.0
        while self.bActive:
            client = None
            try:
                client = ReucamClient(self)
                self.client = client
                client.connect(ReuCameraService.SERVER_ADDR)
                logging.info("ReucamService: connected")
                client.cmd_StartStream(3)  # RAW

                # recv_loop may raise when peer closes; catch inside loop via exception below
                client.recv_loop()

            except Exception:
                # log full traceback (important for debugging 'Peer closed' etc.)
                logging.exception("ReucamService error - reconnecting after failure")
                # ensure we don't keep a broken client reference
                try:
                    if client:
                        client.close()
                except Exception:
                    logging.exception("Error while closing client after failure")
                self.client = None

                # exponential backoff with cap
                time.sleep(backoff)
                backoff = min(backoff * 2, 30.0)
                continue
            else:
                # clean shutdown of recv_loop (no exception) -> reset backoff
                backoff = 1.0
                try:
                    if client:
                        client.close()
                except Exception:
                    logging.exception("Error while closing client in normal shutdown")
                self.client = None
            finally:
                # small delay to avoid tight loop if bActive still True
                time.sleep(0.2)

    def onFrame(self, frame):
        # frame may be None or malformed; guard it
        try:
            if not frame:
                return
            if getattr(frame, 'is_raw', False) and len(frame.data) == FRAME_W * FRAME_H * 3 // 2:
                with self.lock:
                    self.I = frame.data
                    self.ts = frame.timestamp
                    self.frame_event.set()
            else:
                # log unexpected frames at debug level
                logging.debug("Received non-raw or unexpected-size frame")
        except Exception:
            logging.exception("Error in onFrame handler")

    def onFireBorder(self, param1, param2):
        # protect against missing/closed client
        try:
            if self.client is None:
                logging.debug("onFireBorder called but client is None (not connected)")
                return
            # do not call into a client that might be closed; wrap in try
            try:
                self.client.cmd_Notify(REUCAM_NOTIFY_FIREBORDER, param1, param2)
            except Exception:
                logging.exception("Failed to send Notify to reucam client")
        except Exception:
            logging.exception("Unexpected error in onFireBorder")

# -----------------------------
# Auxiliary functions
# -----------------------------
def convert_yuv_to_bgr(yuv_data, width, height):
    y_size = width * height
    uv_size = y_size // 4
    y = np.frombuffer(yuv_data[0:y_size], dtype=np.uint8).reshape((height, width))
    u = np.frombuffer(yuv_data[y_size:y_size+uv_size], dtype=np.uint8).reshape((height//2, width//2))
    v = np.frombuffer(yuv_data[y_size+uv_size:y_size+2*uv_size], dtype=np.uint8).reshape((height//2, width//2))
    u_up = cv2.resize(u, (width, height), interpolation=cv2.INTER_LINEAR)
    v_up = cv2.resize(v, (width, height), interpolation=cv2.INTER_LINEAR)
    yuv = cv2.merge((y, u_up, v_up))
    bgr = cv2.cvtColor(yuv, cv2.COLOR_YUV2BGR)
    return bgr

def get_fft_frequency(signal, fps):
    signal = signal - np.mean(signal)
    fft_vals = np.abs(np.fft.rfft(signal))[1:]
    f_axis = np.linspace(0, fps / 2, len(fft_vals) + 1)[1:]
    if len(fft_vals) == 0:
        return 0
    peaks, _ = find_peaks(fft_vals, height=np.max(fft_vals))
    if len(peaks) > 0 and fft_vals[peaks[0]] > fourier_peak_threshold * np.median(fft_vals):
        return f_axis[peaks[0]]
    return 0

def process_box(frame, gray, B2, h, w):
    b2 = B2[h*BOX_SIZE:(h+1)*BOX_SIZE, w*BOX_SIZE:(w+1)*BOX_SIZE]

    # FFT processing
    z = np.roll(fourier_buff[:, h, w], 1)
    z[0] = np.sum(b2)
    fourier_buff[:, h, w] = z

    # FFT
    freq = get_fft_frequency(fourier_buff[:, h, w], FPS)
    
    if abs(freq - LED_FREQ) < FREQ_TOL:
        freq_counter[h, w] += 1
        x1 = w * BOX_SIZE
        y1 = h * BOX_SIZE
        x2 = min((w + 1) * BOX_SIZE, frame_width)
        y2 = min((h + 1) * BOX_SIZE, frame_height)
        return (x1, y1, x2, y2), freq

    freq_counter[h, w] = 0
    return None, None

def process_flicker_pixels(frame, roi, flicker_buff, frame_count, buffer_len=FLICKER_BUFFER_LENGTH, thresh=FLICKER_DIFF_THRESHOLD):
    global flicker_stable_count, mode, bbox_mode3
    if roi is None:
        return None, flicker_buff, frame_count, bbox_mode3

    x1, y1, x2, y2 = roi
    x1 = max(0, x1)
    y1 = max(0, y1)
    x2 = min(frame.shape[1]-1, x2)
    y2 = min(frame.shape[0]-1, y2)

    if x2 <= x1 or y2 <= y1:
        return None, flicker_buff, frame_count, bbox_mode3
    
    roi_mode2 = frame[y1:y2, x1:x2]   
    
    # HSV COLOR FILTER CODE - use global parameters
    hsv = cv2.cvtColor(roi_mode2, cv2.COLOR_BGR2HSV)
    
    mask_hsv = cv2.inRange(hsv, HSV_LOWER_BOUND, HSV_UPPER_BOUND)  # Use global HSV bounds
    filtered = cv2.bitwise_and(roi_mode2, roi_mode2, mask=mask_hsv)
         
    roi_gray = cv2.cvtColor(filtered, cv2.COLOR_BGR2GRAY)
    
    h, w = roi_gray.shape

    if flicker_buff is None or flicker_buff.shape[1:] != (h, w):
        flicker_buff = np.zeros((buffer_len, h, w), dtype=np.uint8)
        frame_count = 0
        flicker_stable_count = 0

    flicker_buff = np.roll(flicker_buff, -1, axis=0)
    flicker_buff[-1] = roi_gray
    frame_count += 1

    mask = np.zeros((h, w), dtype=np.uint8)
    if frame_count > buffer_len:
        diff = flicker_buff.max(axis=0) - flicker_buff.min(axis=0)
        mask[diff > thresh] = 200
        print('max_diff',np.max(diff))
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            all_points = np.vstack(contours)
            x, y, w_box, h_box = cv2.boundingRect(all_points)

            x_margin = max(0, x1 + x - MARGIN3_SAFETY_FACTOR)  # Use global margin parameter
            y_margin = max(0, y1 + y - MARGIN3_SAFETY_FACTOR)
            x2_margin = min(frame.shape[1], x1 + x + w_box + MARGIN3_SAFETY_FACTOR)
            y2_margin = min(frame.shape[0], y1 + y + h_box + MARGIN3_SAFETY_FACTOR)
            cv2.rectangle(frame, (x1+x, y1+y), (x1+x+w_box, y1+y+h_box), CONTOUR_BOX_COLOR, CONTOUR_BOX_THICKNESS)  # Use global parameters
            
            flicker_stable_count += 1
            
            if flicker_stable_count >= STABLE_FRAMES_THRESHOLD_MODE2:  # Use global parameter
                bbox_mode3 = (x_margin, y_margin,x2_margin - x_margin, y2_margin - y_margin)
                tracker.init(frame, bbox_mode3)
                logging.info("tracker.init(frame, bbox_mode3)")
                logging.info(f"bbox_mode3={bbox_mode3}")
                mode = 3
        else:
            flicker_stable_count = 0
            bbox_mode3 = None
            mode = 1

    return mask, flicker_buff, frame_count, bbox_mode3

def convert_bboxes_to_reucam_params(bbox):
    """
    Converts a bounding box to ReuCam parameters safely.
    bbox = [x0, y0, x1, y1]
    Returns param1, param2 (packed)
    """

    # Scale coordinates to FRAME_W / FRAME_H
    x0 = math.ceil(bbox[0] * FRAME_W / frame_width)
    x1 = math.ceil(bbox[2] * FRAME_W / frame_width)
    y0 = math.ceil(bbox[1] * FRAME_H / frame_height)
    y1 = math.ceil(bbox[3] * FRAME_H / frame_height)

    w = x1 - x0
    h = y1 - y0

    # If frame is large, reduce w/h
    if frame_width > 640:
        w = math.ceil(w / 2)
        h = math.ceil(h / 2)

    # Clamp values to valid ranges
    x0 = max(0, min(65535, x0))
    y0 = max(0, min(65535, y0))  # ushort range
    w = max(0, min(255, w))      # B range
    h = max(0, min(255, h))      # B range

    param1 = x0
    try:
        param2 = struct.pack('HBB', y0, w, h)
    except struct.error as e:
        logging.error(f"struct.pack failed: y0={y0}, w={w}, h={h}")
        param2 = struct.pack('HBB', 0, 0, 0)

    param2 = struct.unpack('i', param2)[0]

    return param1, param2

# Set stdin to non-blocking once at startup
fd = sys.stdin.fileno()
fl = fcntl.fcntl(fd, fcntl.F_GETFL)
fcntl.fcntl(fd, fcntl.F_SETFL, fl | os.O_NONBLOCK)

def check_stdin():
    try:
        line = sys.stdin.readline()
        if line:
            return line.strip()
    except Exception:
        return None
    return None

last_fireborder = None
last_fire_time = 0

# -----------------------------
# Main
# -----------------------------
if __name__ == "__main__":
    reucam = ReucamService()
    reucam.start()
    def cleanup():
        try:
            if is_recording:
                stop_recording()
            reucam.onFireBorder(0, 0)
        except Exception:
            pass  # ignore errors if reucam is not available yet
   
    # Register cleanup for normal exit
    atexit.register(cleanup)
   
    # Handle termination signals
    def handle_signal(signum, frame):
        cleanup()
        sys.exit(0)
   
    signal.signal(signal.SIGINT, handle_signal)   # CTRL+C
    signal.signal(signal.SIGTERM, handle_signal)  # kill command
    signal.signal(signal.SIGPIPE, signal.SIG_DFL) # keep default SIGPIPE behavior

    I = None
    frame_count = 0
    fps_start_time = time.time()
    
    while True:
        if reucam.frame_event.wait(FRAME_TIMEOUT_SECONDS):  # Use global timeout parameter
            break
        logging.info("Waiting for camera to connect...")

    track_fail_time = None  # time of first track failure
    tracker = cv2.TrackerCSRT_create()
    
    # Print instructions for video recording
    print("=== VIDEO RECORDING COMMANDS ===")
    print("Type 'S' to start RAW video recording")
    print("Type 'P' to stop RAW video recording")
    print("==================================")
    
    while True:
        line = check_stdin()
        
        # VIDEO RECORDING COMMANDS
        if line == "s":
            start_recording()
        elif line == "p":
            stop_recording()
        elif line == "RESET_BORDER":
            logging.info("Received RESET_BORDER command")
            reucam.onFireBorder(0, 0)
            mode = 1
            bbox_mode3 = None
            flicker_stable_count = 0
            roi_stable_count = 0
            fourier_buff[:] = 0
   
        reucam.frame_event.clear()
        
        # If current frame equals to previous frame or its None - continue
        if reucam.I == I:
            continue
        I = reucam.I
        ts = reucam.ts

        frame_count += 1
        current_time = time.time()
        if current_time - fps_start_time >= FPS_CALCULATION_INTERVAL:  # Use global parameter for FPS calculation
            fps = frame_count / (current_time - fps_start_time)
            logging.info(f"FPS: {fps:.2f}")
            frame_count = 0
            fps_start_time = current_time
            FPS=fps
       
        if I is None:
            logging.warning("Frame is of type None, skipping.")
            continue

        yuv_data = reucam.I
        
        # ===== RECORD RAW FRAME (FIRST, BEFORE ANY PROCESSING) =====
        record_raw_frame(yuv_data)

        # ===== MODE 1 OPTIMIZATION: Use Y channel directly for frequency detection =====
        # For Mode 1, extract Y channel directly from YUV data to avoid expensive color conversions
        y_size = FRAME_W * FRAME_H
        gray = np.frombuffer(yuv_data[0:y_size], dtype=np.uint8).reshape((FRAME_H, FRAME_W))
        _, B2 = cv2.threshold(gray, BINARY_THRESHOLD_MODE1, 255, cv2.THRESH_BINARY)  # Use global threshold parameter
        # Create a simple BGR frame for visualization (Y channel only)
        frame = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
        frame[CLOCK_REGION_Y1:CLOCK_REGION_Y2, CLOCK_REGION_X1:CLOCK_REGION_X2, :] = 0  # Remove clock using global parameters

        if mode == 1:
            logging.info("mode==1")
            if track_fail_time is not None:
                logging.info(f"current_time - track_fail_time = {current_time - track_fail_time}")
            current_time = time.time()
            if track_fail_time is not None and current_time - track_fail_time >= TRACKER_TIMEOUT_SECONDS:  # Use global parameter
                print("1 0 0", flush=True)
                logging.info("mode 1: 1 0 0")
            elif track_fail_time is None:
                print("1 0 0", flush=True)
                logging.info("mode 1: 1 0 0")
           
            found_boxes = []
            for h in range(HE):
                for w in range(WI):
                    cv2.rectangle(frame,
                                  (w*BOX_SIZE, h*BOX_SIZE),
                                  (w*BOX_SIZE+BOX_SIZE, h*BOX_SIZE+BOX_SIZE),
                                  GRID_COLOR_MODE1, GRID_LINE_THICKNESS)  # Use global grid parameters

                    box, freq = process_box(frame, gray, B2, h, w)

                    if box is not None:
                        found_boxes.append(box)

            if found_boxes:
                x1 = max(0, min(b[0] for b in found_boxes) - MARGIN)
                y1 = max(0, min(b[1] for b in found_boxes) - MARGIN)
                x2 = min(frame_width,  max(b[2] for b in found_boxes) + MARGIN)
                y2 = min(frame_height, max(b[3] for b in found_boxes) + MARGIN)

                cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,255), 2)

                roi_stable_count += 1
                if roi_stable_count >= ROI_STABLE_FRAMES:
                    bbox_mode2 = (x1 - MARGIN, y1 - MARGIN, x2 + MARGIN, y2 + MARGIN)
                    top_point = (x1- MARGIN, y1- MARGIN)
                    bottom_point = (x2+ MARGIN, y2+ MARGIN)
                    mode = 2
            else:
                roi_stable_count = 0
   
        elif mode == 2:
            frame = convert_yuv_to_bgr(yuv_data, FRAME_W, FRAME_H)
            frame[CLOCK_REGION_Y1:CLOCK_REGION_Y2, CLOCK_REGION_X1:CLOCK_REGION_X2, :] = 0  # Remove clock using global parameters
            logging.info("mode==2")  
            current_time = time.time()
            if track_fail_time is not None:
                logging.info(f"current_time - track_fail_time = {current_time - track_fail_time}")
            if track_fail_time is not None and current_time - track_fail_time >= TRACKER_TIMEOUT_SECONDS:  # Use global parameter
                print("1 0 0", flush=True)
                logging.info("mode 2: 1 0 0")
            elif track_fail_time is None:
                print("1 0 0", flush=True)
                logging.info("mode 2: 1 0 0")

            if bbox_mode2 is not None:
                x1, y1, x2, y2 = bbox_mode2
            mask, flicker_buff, frame_idx,bbox_mode3 = process_flicker_pixels(frame, bbox_mode2, flicker_buff, frame_idx)
            if bbox_mode3:
                (x0, y0, w0, h0) = [int(v) for v in bbox_mode3]

        elif mode == 3:
            cross_conf = configparser.ConfigParser()
            cross_conf.read(cross_conf_path)
            center_x0 = cross_conf.getint("camera", "cross_w_pos", fallback=frame_width // 2)
            center_y0 = cross_conf.getint("camera", "cross_h_pos", fallback=frame_height // 2)
            logging.info("mode==3")
            if bbox_mode3 is not None:
                try:
                    logging.info(f"Frame shape: {frame.shape}, bbox_mode3: {bbox_mode3}")
                except Exception as e:
                    logging.error(f"Frame info error: {e}")
                   
                x, y, w, h = bbox_mode3
                if x < 0 or y < 0 or x + w > frame.shape[1] or y + h > frame.shape[0]:
                    logging.error(f"Invalid bbox before tracker.update(): {bbox_mode3}")
                    success = False
                    continue

                try:
                    logging.info("tracker.update()")
                    success, bbox = tracker.update(frame)
                except Exception as e:
                    logging.exception(f"tracker.update() failed: {e}")
                    success = False

                logging.info("if bbox_mode3 is not None:")
                if success:    
                    track_fail_time = None    # reset the fail timer because tracking succeeded
                    logging.info(f"Frame shape: {frame.shape}, bbox: {bbox}")
                    x, y, w, h = [int(v) for v in bbox]
                    x = max(0, x)
                    y = max(0, y)
                    w= min(frame.shape[1]-1-x, w)
                    h = min(frame.shape[0]-y-1, h)
                    bbox_debug=(x,y,w,h)
                    
                    # DISTANCE CALCULATION AND SAFETY CHECK
                    Distance=(LED_CIRCLE_SIZE/(2*w*np.tan(np.deg2rad(0.5*IFOV_x))))/MARGIN3_factor
                    print('Distance', Distance)
                    if Distance < DISTANCE_SAFETY_CHECK:  # Use global safety check parameter
                        reucam.onFireBorder(0, 0)
                        mode = 1
                        tcounter = 0
                        failure_count_m3 = 0
                        pixel_count_buff[:] = 0
                        flicker_stable_count = 0
                        bbox_mode3 = None        
                        flicker_buff = None      
                        roi_stable_count = 0    
                        fourier_buff = np.zeros((CALC_LENGTH, HE, WI))
                                       
                    logging.info(f"Frame shape: {frame.shape}, bbox_d: {bbox}")
                    
                    center_x = (x + w // 2)
                    center_y = (y + h // 2)
                    bbox_draw=[x,y,x+w,y+h]
                    param1, param2 = convert_bboxes_to_reucam_params(bbox_draw)
                   
                    # inside mode 3 loop
                    param1, param2 = convert_bboxes_to_reucam_params(bbox_draw)
                    current_time = time.time()
                    if last_fireborder != (param1, param2) or current_time - last_fire_time > CROSS_HAIR_UPDATE_INTERVAL:  # Use global parameter
                        reucam.onFireBorder(0,0)
                        time.sleep(0.01)
                        reucam.onFireBorder(param1, param2)
                        last_fireborder = (param1, param2)
                        last_fire_time = current_time

                    x_alignment=center_x-center_x0
                    y_alignment = center_y - center_y0
                    y_alignment_deg=-2*np.round((center_x-center_x0)*IFOV_x,1)      # transmit to Israel with a negative sign
                    y_alignment_deg = 0 if abs(y_alignment_deg) < ALIGNMENT_ANGLE_THRESHOLD else y_alignment_deg  # Use global parameter
                    x_alignment_deg = -2*np.round((center_y - center_y0)*IFOV_y,1)  # transmit to Israel with a negative sign
                    x_alignment_deg = 0 if abs(x_alignment_deg) < ALIGNMENT_ANGLE_THRESHOLD else x_alignment_deg  # Use global parameter
                   
                    logging.info(f"x={x_alignment_deg} y={y_alignment_deg}")
                    try:
                        print(f"0 {x_alignment_deg} {y_alignment_deg}", flush=True)
                    except BrokenPipeError:
                        # just ignore the error and continue running
                        sys.stderr.write("stdout closed, ignoring...\n")

                    if (np.abs(x_alignment_deg) > ALIGNMENT_DEADBAND) or (np.abs(y_alignment_deg) > ALIGNMENT_DEADBAND):  # Use global parameter
                        color=(0,0,255)
                    else:
                        color=(255,0,0)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), TRACKING_BOX_COLOR, TRACKING_BOX_THICKNESS, 1)  # Use global parameters
                    center_text2= f"({x_alignment_deg}, {y_alignment_deg})"
                    center_text = f"({x_alignment}, {y_alignment})"
                    center_text3= f"({center_x}, {center_y})"
                    text_position = (x, y + h + 20)
                    text2_position = (x, y + h + 40)
                    cv2.putText(frame, center_text2, text_position, cv2.FONT_HERSHEY_SIMPLEX, ALIGNMENT_TEXT_SCALE, color, 2)  # Use global text scale
                    cv2.rectangle(frame, (x, y), (x+w, y+h), TRACKING_BOX_COLOR, TRACKING_BOX_THICKNESS)  # Use global parameters

                    roi_binary = B2[y:y+h, x:x+w]
                    non_zero_pixel_count = np.sum(roi_binary)
                    pixel_count_buff = np.roll(pixel_count_buff, -1)
                    pixel_count_buff[-1] = non_zero_pixel_count
                    tcounter += 1
                    if tcounter > CALC_LENGTH:
                        detected_freq = get_fft_frequency(pixel_count_buff, FPS)

                        if abs(detected_freq - LED_FREQ) > FREQ_TOL:
                            failure_count_m3 += 1
                            if failure_count_m3 >= MAX_FAILURES_M3:
                                logging.info('Freq failed')
                                reucam.onFireBorder(0, 0)
                                mode = 1
                                tcounter = 0
                                failure_count_m3 = 0
                                pixel_count_buff[:] = 0
                                flicker_stable_count = 0
                                bbox_mode3 = None        
                                flicker_buff = None      
                                roi_stable_count = 0    
                                fourier_buff = np.zeros((CALC_LENGTH, HE, WI))
                        else:
                            failure_count_m3 = 0
                else:
                    logging.info('Track failed')
                    roi_stable_count = 0
                    reucam.onFireBorder(0, 0)
                    mode = 1
                    flicker_stable_count = 0
                    bbox_mode3 = None
                    failure_count_m3 = 0
                    roi_stable_count = 0
                    fourier_buff = np.zeros((CALC_LENGTH, HE, WI))
                   
                    current_time = time.time()
                    if track_fail_time is None:
                        # first failure, start the timer
                        track_fail_time = current_time
                        logging.info('Track failed - starting timer')
                    elif current_time - track_fail_time >= TRACKER_TIMEOUT_SECONDS:  # Use global parameter
                        logging.info('else mode 3: Track failed')
                        print("1 0 0", flush=True)
            else:
                print("1 0 0", flush=True)
       
        else:
            print("1 0 0", flush=True)
   
        cv2.putText(frame, f"Mode: {mode}", MODE_TEXT_POSITION,  # Use global parameters
                    cv2.FONT_HERSHEY_SIMPLEX, MODE_TEXT_SCALE, MODE_TEXT_COLOR, MODE_TEXT_THICKNESS)

        # Display recording status on frame using global parameters
        if is_recording:
            cv2.putText(frame, f"REC: {frame_count_record} frames", RECORDING_TEXT_POSITION,
                       cv2.FONT_HERSHEY_SIMPLEX, RECORDING_TEXT_SCALE, RECORDING_TEXT_COLOR, RECORDING_TEXT_THICKNESS)
        
        key = cv2.waitKey(1) & 0xFF
        if key == 27 or key == ord('q'):
            break

    cv2.destroyAllWindows()